In [49]:
! mkdir include
! mkdir src
! mkdir bin
! mkdir csv

In [50]:
%%file Makefile

#compilare con:
#make dev VERSION=1_1

#compilers
NVCC=nvcc
GXX=g++

#directories
src_path=src
header_path=include

dev:
	@echo "version:" $(VERSION)
	nvcc -rdc=true -o bin/fwa_dev_v_$(VERSION).out \
		device_floyd_warshall_v_$(VERSION).cu \
		src/adj_matrix_utils.cu \
		src/adj_matrix_utils.cpp \
		src/cuda_errors_utils.cu \
		src/generate_n_b_couples.cpp \
		src/handle_arguments_and_execute.cu \
		src/host_floyd_warshall.cpp \
		src/performance_test.cu \
		src/statistical_test.cpp

read_matrix :	
	nvcc -rdc=true -o bin/read_matrix.out \
		main.cpp \
		src/adj_matrix_reader.cpp \
		src/adj_matrix_utils.cpp

fwm:
	g++ -o bin/fwm.out \
		host_floyd_warshall_matrix.cpp \
		src/adj_matrix_utils.cpp  \
		src/host_floyd_warshall.cpp \

fwa:
	g++ -o bin/fwa.out \
		host_floyd_warshall_array.cpp \
		src/adj_matrix_utils.cpp \
		src/host_floyd_warshall.cpp

dev_test:
	nvcc -rdc=true -o bin/dev_test.out \
		device_test.cu \
		src/cuda_errors_utils.cu

generate_n_b:
	g++ -o bin/generate_and_print_n_b.out \
		generate_and_print_n_b.cpp \
		src/generate_n_b_couples.cpp

In [51]:
%%file include/adj_matrix_reader.hpp

#ifndef ADJ_MATRIX_READER_HPP
#define ADJ_MATRIX_READER_HPP

#include <string>

/// <summary>
/// Leggi un file CSV contenente una matrice di adiacenza.
/// </summary>
/// <param name="filename">Il nome del file da leggere</param>
/// <param name="delim">Il delimitatore del file CSV</param>
/// <param name="adjMatrix">Un puntatore vuoto dove verr� allocata la memoria per mem. la matrice</param>
/// <param name="numberOfNodes">Il numero di nodi del grafo (corrisponde al numero di righe e colonne della matrice)</param>
/// <return>0 if okay, else something less</return>
int** readAdjMatrixCSV(std::string filename, const char delim, int* numberOfNodes);

#endif


In [52]:
%%file include/adj_matrix_utils.cuh

#ifndef ADJ_MATRIX_UTILS_CUH
#define ADJ_MATRIX_UTILS_CUH

//  PRINT UTILS

#include "macros.hpp"

__device__ void print_matrix_device(int *matrix, int m, int n, int pitch);


#endif // ADJ_MATRIX_UTILS_CUH

In [53]:
%%file include/adj_matrix_utils.hpp

#ifndef ADJ_MATRIX_UTILS_HPP
#define ADJ_MATRIX_UTILS_HPP

#include <stdbool.h>

#include "macros.hpp"

/// Parameters used when generating a graph
#define DENSITY 60
#define MIN_COST 1
#define MAX_COST 20

// ---------------------------------------------------------------
//  PRINT UTILS

void print_array(int *array, int size);
void print_matrix(int **matrix, int m, int n);
void print_element(int val, int infinity);
void print_arr_matrix(int *matrix, int m, int n);

// ---------------------------------------------------------------
// MATRIX GENERATION, COMPARE and others utils

int** allocate_matrix(int m, int n);
void populate_adj_matrix(int **matrix, int n, int seed, bool oriented_graph);
bool same_matrixes(int **matrix_1, int **matrix_2, int m, int n, bool oriented_graph);

// ---------------------------------------------------------------
// ARRAY MATRIX FUNCTIONS VARIANTS

int* allocate_arr_matrix(int m, int n);
void populate_arr_adj_matrix(int* arr_matrix, int n, int seed, bool oriented_graph);
bool same_arr_matrixes(int *matrix_1, int *matrix_2, int m, int n, bool oriented_graph);

#endif // ADJ_MATRIX_UTILS_H

In [54]:
%%file include/cuda_errors_utils.cuh

#ifndef CUDA_ERRORS_UTILS_CUH
#define CUDA_ERRORS_UTILS_CUH


#define HANDLE_ERROR(err) (handle_error(err, __FILE__, __LINE__))

void handle_error(cudaError_t err, const char *file, int line);

void check_CUDA_error(const char *msg);

#endif

In [55]:
%%file include/generate_n_b_couples.hpp

#ifndef GENERATE_N_B_COUPLES
#define GENERATE_N_B_COUPLES

// c++ libraries
// for std::vector
#include <vector>
// for std::pair
#include <utility>
// for std::string
#include <string>

// generate the list of couples (n,b)
std::vector<std::pair<int, int>> generate_list_of_all_n_b(int min_input_size, int max_input_size, int max_num_of_b_per_n, double to_multiply, int to_sum, int min_blocking_factor, int max_num_tests, int seed);

// prints to file the list of couples (n,b)
void print_list_to_file(std::vector<std::pair<int, int>> list_of_all_n_b, std::string filename);

// MACROS 

// DEFAULT VALUES:
// returns a random number between 1.300 and 1.600
#define OBTAIN_VAL_TO_MULTIPLY(rand_value) ((double) ((rand_value % 300) + 1300)) / ((double) 1000)
// outputs a random even number between 0 and 100
#define OBTAIN_VAL_TO_SUM(rand_value) ((rand_value % 101) * 2)

#endif //GENERATE_N_B_COUPLES


In [56]:
%%file include/handle_arguments_and_execute.cuh

#ifndef HANDLE_ARGUMENTS_AND_EXECUTE
#define HANDLE_ARGUMENTS_AND_EXECUTE

int handle_arguments_and_execute(int argc, char *argv[], void (*f) (int* arr_matrix, int n, int b));

#endif //HANDLE_ARGUMENTS_AND_EXECUTE


In [57]:
%%file include/host_floyd_warshall.hpp

#ifndef HOST_FLOYD_WARSHALL
#define HOST_FLOYD_WARSHALL


#include "macros.hpp"

// ---------------------------------------------------------------------------
// Matrix data structure version

void host_matrix_floyd_warshall(int **matrix, int n);
void host_matrix_floyd_warshall_blocked(int **matrix, int n, int B);
void host_matrix_execute_round(int **matrix, int n, int t, int row, int col, int B);

// ---------------------------------------------------------------------------
// Array data structure version

void host_array_floyd_warshall(int *matrix, int n);
void host_array_floyd_warshall_blocked(int *matrix, int n, int B);
void host_array_execute_round(int *matrix, int n, int t, int row, int col, int B);

#endif

In [58]:
%%file include/include_needed_libraries.cuh

// c libraries
#include <stdio.h>

// c++ libraries
// for assertions
#include <cassert>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include "adj_matrix_utils.cuh"
#include "adj_matrix_utils.hpp"
#include "cuda_errors_utils.cuh"
#include "handle_arguments_and_execute.cuh"
#include "host_floyd_warshall.hpp"
#include "macros.hpp"
#include "performance_test.cuh"
#include "statistical_test.hpp"

In [59]:
%%file include/lcm.hpp

#ifndef LCM_HPP
#define LCM_HPP

int lcm(int x, int y) {

    int r=0;

    while (r%x!=0 || r%y!=0) {
        r++;
    }
    
    return r;
}

#endif

In [60]:
%%file include/macros.hpp

#ifndef MACROS_HPP
#define MACROS_HPP

/// Big M, value that should be threated as "infinity"
#define INF __INT16_MAX__

/// Get minimum or maximum of two values
// renamed from min and max to avoid overload with std::mmin and std::mmax
#define mmin(a,b) ((a < b) ? a : b)
#define mmax(a,b) ((a > b) ? a : b)

/// Sum two numbers if they are not infinite, else return infinity
#define sum_if_not_infinite(x1,x2,infinity) ((x1==infinity) || (x2==infinity)) ? infinity : (x1+x2)


#define MAX_BLOCK_SIZE 1024 // in realtà basta fare le proprerties della macchina
#define MAX_BLOCKING_FACTOR 32 // 32*32 = 1024

/// Macro to get block starting position (of a column or of a row)
#define BLOCK_START(block_index,B) (block_index * B)

/// Macro to get block ending position (of a column or of a row)
#define BLOCK_END(block_index,B) ((block_index+1) * B)

/// Print a bool as a string
#define bool_to_string(cond) (cond ? "true" : "false")

/// returns the pointer of given (i,j) using the access pattern of a 2D pitched memory
#define pitched_pointer(matrix, i, j, pitch) ( (int *) (((char*) matrix + i * pitch) + j) )


#endif

In [61]:
%%file include/performance_test.cuh

#ifndef PERFORMANCE_TEST_CUH
#define PERFORMANCE_TEST_CUH

void do_nvprof_performance_test(void (*floyd_warshall_arr_algorithm)(int * matrix, int n, int B), int input_size, int blocking_factor, int number_of_tests, int seed);

#endif

In [62]:
%%file include/statistical_test.hpp

#ifndef STATISTICAL_TEST_HPP
#define STATISTICAL_TEST_HPP

#define RANDOM_SEED     0
#define RANDOM_CONSTANT -1


struct MultiSizeTestParameters {
    void (*f) (int* arr_matrix, int n, int b) = NULL;   // test function
    void (*g) (int* arr_matrix, int n, int b) = NULL;   // compare function
    int start_input_size        = 4;                    // min n to test
    int end_input_size          = 1024;                 // max n to test
    double to_multiply          = RANDOM_CONSTANT;      // used to linearly increase input size: { next_size = cur_size * to_multiply + to_sum } (pass RANDOM_CONSTANT to generate a random value)
    int to_sum                  = RANDOM_CONSTANT;      // used to linearly increase input size: { next_size = cur_size * to_multiply + to_sum } (pass RANDOM_CONSTANT to generate a random value)
    int seed                    = RANDOM_SEED;          // seed for input generation (pass RANDOM_SEED to generate a random seed)
    int n_tests_per_round       = 500;                  // number of different test foreach given couple (n,B)
    int print_progress_perc     = 1;                    // print progress of a test for a given couple (n,B) (for ex. 4 ==> 100/4 = 25% ==> print progress at 25%, 50%, 75%), if 1 is disabled
    bool stop_current_if_fail   = true;                 // true ==> if found an error for a given couple (n,B): stop test this couple but keep testing other couples
    bool stop_all_if_fail       = false;                // true ==> if found an error for a given couple (n,B): stop all tests (return control) (NB: stop_all_if_fail ==> stop_current_if_fail but not viceversa)
    bool print_failed_tests     = true;                 // true ==> if found an error print seed and the index of the test
    int min_blocking_factor     = 2;                    // the minimum blocking factor you are intrested testing
};

// generates many couples (n,B) and foreach couple generates inputs and executes executes f,g n_tests_per_round times
// returns: total number of errors of all couples
int  multi_size_statistical_test(MultiSizeTestParameters params);
void print_multi_size_test_parameters(MultiSizeTestParameters params);

// -------------------------------------------------------------------------------------------------

struct CallSingleSizeTestParameters {
    void (*f) (int* arr_matrix, int n, int b) = NULL;   // test function
    void (*g) (int* arr_matrix, int n, int b) = NULL;   // compare function
    int input_size              = 256;                  // input test size
    int blocking_factor         = 12;                   // blocking factor test size
    int seed                    = RANDOM_SEED;          // seed for input generation (pass RANDOM_SEED to generate a random seed)
    int n_tests                 = 500;                  // number of different tests to do
    int print_progress_perc     = 1;                    // print progress of a test for a given couple (n,B) (for ex. 4 ==> 100/4 = 25% ==> print progress at 25%, 50%, 75%), if 1 is disabled
    bool stop_current_if_fail   = true;                 // true ==> if found an error: stop testing (return control)
    bool print_failed_tests     = true;                 // true ==> if found an error print seed and the index of the test
};

// given a couple (n,B), generates inputs and executes f,g n_tests_per_round times
// returns: total number of errors of the given couple
int  call_single_size_statistical_test(CallSingleSizeTestParameters params);
void print_call_single_size_statistical_test_parameters(CallSingleSizeTestParameters params);
    
// -------------------------------------------------------------------------------------------------

struct ExecSingleSizeTestParameters {
    void (*f) (int* arr_matrix, int n, int b) = NULL;   // test function
    void (*g) (int* arr_matrix, int n, int b) = NULL;   // compare function
    int input_size              = 256;                  // input test size
    int blocking_factor         = 12;                   // blocking factor test size
    int *input_instance         = NULL;                 // input instance populated
};

// given a couple (n,B) and an an input instance populated executes f,g 1 time
// returns: f(input) == g(input)
bool exec_single_single_statistical_test(ExecSingleSizeTestParameters params);
void print_exec_single_size_statistical_test_parameters(ExecSingleSizeTestParameters params);

#endif

In [63]:
# END HEADER REGION
# ----------------------------------------------------------------------------------------------------------------------------------------
# START SRC REGION

In [64]:
%%file src/adj_matrix_reader.cpp

#include <fstream>
#include <sstream>
#include <iostream>

#include "../include/adj_matrix_reader.hpp"

int _getNumberOfNodes(std::string adjMatrixLine, const char delim) {

	// insipired to: https://java2blog.com/split-string-space-cpp/#Using_getline_Method

	std::istringstream ss(adjMatrixLine);

	int nodesCounter = 0;

	std::string s;
	while (std::getline(ss, s, delim)) {
		nodesCounter++;
	}

	return nodesCounter;
}

int _parseLine(std::string adjMatrixLine, const char delim, int lineNumber, int** adjMatrix) {

	// insipired to: https://java2blog.com/split-string-space-cpp/#Using_getline_Method

	std::istringstream ss(adjMatrixLine);
	std::string itemStr;

	int i = 0;
	while (std::getline(ss, itemStr, delim)) {

		int value = std::stoi(itemStr);
		adjMatrix[lineNumber][i] = value;

		i++;
	}

	return 0;
}


int** readAdjMatrixCSV(const std::string filename, const char delim, int *numberOfNodes) {

	std::ifstream fs(filename);
	
	if (!fs.is_open()) {
		// todo: add error
	}
	
	if (fs.eof()) {
		// todo: add error
	}

	// read first line
	std::string line;
	std::getline(fs, line);
	int lineNumber = 0;
	
	// get number of nodes
	*numberOfNodes = _getNumberOfNodes(line, delim);

	// allocate memory for matrix
	int** adjMatrix = (int **) malloc(sizeof(int*) * (*numberOfNodes));

	// parse all lines and fill adjMatrix
	do {
		adjMatrix[lineNumber] = (int*) malloc(sizeof(int) * (*numberOfNodes));
		_parseLine(line, delim, lineNumber, adjMatrix);
		lineNumber++;
	} while (std::getline(fs, line));

	return adjMatrix;
}


In [65]:
%%file src/adj_matrix_utils.cpp


#include <stdio.h>
#include <stdlib.h>

#include "../include/adj_matrix_utils.hpp"

// ---------------------------------------------------------------
//  PRINT UTILS

void print_matrix(int **matrix, int m, int n) {
    printf("[\n");
    for (int i = 0; i < m; i++) {
        printf("  ");
        print_array(matrix[i], n);
    }
    printf("]\n");
}

void print_array(int *array, int size) {
    printf("[");
    for (int i = 0; i < size; i++) {
        print_element(array[i], INF);
        if (i < size-1) printf(", ");
    }
    printf("]\n");
}

void print_element(int val, int infinity) {
    if (val < infinity)
        printf("%02d", val);
    else 
        printf("--");
}

// ---------------------------------------------------------------
// MATRIX GENERATION, COMPARE and others utils

bool same_matrixes(int **matrix_1, int **matrix_2, int m, int n, bool oriented_graph) {
    for (int i = 0; i < m; i++) {
        for (int j = oriented_graph ? 0 : (i+1); j < n; j++) {
            if(matrix_1[i][j] != matrix_2[i][j]) return false;
        }
    }
    return true;
}

int** allocate_matrix(int m, int n) {
    //matrix with row major order:
    //m rows pointers
    int** matrix = (int **) malloc(sizeof(int *) * m);
    for (int i = 0; i < m; i++) {
        //each row has n intengers
        matrix[i] = (int *) malloc(sizeof(int) * n);
    }
    return matrix;
}

void populate_adj_matrix(int **matrix, int n, int seed, bool oriented_graph) {
    for (int i = 0; i < n; i++) {
        //diagonal always zero (distance 0 to myself)
        matrix[i][i] = 0;
        for (int j = oriented_graph ? 0 : (i+1); j < n; j++) {
            if (i != j) {               
                bool add_edge = (rand() % 100) <= DENSITY;
                int val = (rand() % MAX_COST) + MIN_COST;
                matrix[i][j] = add_edge ? val : INF;
                if (! oriented_graph) {
                    //non-oriented graph
                    matrix[j][i] = matrix[i][j];
                }        
            }
        }
    }
}

// ---------------------------------------------------------------
// ARRAY MATRIX FUNCTIONS VARIANTS

void print_arr_matrix(int *matrix, int m, int n) {
    printf("[\n");
    for (int i = 0; i < m; i++) {
        printf("  ");
        print_array(&(matrix[i*n]), n);
    }
    printf("]\n");
}

bool same_arr_matrixes(int *matrix_1, int *matrix_2, int m, int n, bool oriented_graph) {
    for (int i = 0; i < m; i++) {
        for (int j = oriented_graph ? 0 : (i+1); j < n; j++) {
            if(matrix_1[i*n + j] != matrix_2[i*n + j]) return false;
        }
    }
    return true;
}

int* allocate_arr_matrix(int m, int n) {
    return (int *) malloc(sizeof(int) * m * n);
}

void populate_arr_adj_matrix(int* arr_matrix, int n, int seed, bool oriented_graph) {
    for (int i = 0; i < n; i++) {
        arr_matrix[i*n + i] = 0;
        for (int j = oriented_graph ? 0 : (i+1); j < n; j++) {
            if (i != j) {        
                bool add_edge = (rand() % 100) <= DENSITY;
                int val = (rand() % MAX_COST) + MIN_COST;
                arr_matrix[i*n + j] = add_edge ? val : INF;
                if (! oriented_graph) {
                    //non-oriented graph
                    arr_matrix[j*n + i] = arr_matrix[i*n + j];
                }
            }
        }
    }
}

In [66]:
%%file src/adj_matrix_utils.cu


#include <stdio.h>
#include <stdlib.h>

#include "../include/adj_matrix_utils.cuh"

__device__ void print_matrix_device(int *matrix, int m, int n, int pitch) {
    printf("[\n");
    for (int i = 0; i < m; i++) {
        printf("  ");
        for (int j = 0; j < n; j++) {
            int val;
            if (pitch == 0) {
                val = matrix[i*n + j];
            } else {
                int* temp = pitched_pointer(matrix, i, j, pitch);
                val = *temp;
            }
            if (val < INF)
                printf("%02d", val);
            else 
                printf("--");
            if (j < n-1) printf(", ");
        }
        printf("\n");
    }
    printf("]\n");
}


In [67]:
%%file src/cuda_errors_utils.cu

#include <stdio.h>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include "../include/cuda_errors_utils.cuh"

void handle_error(cudaError_t err, const char *file, int line) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ), file, line );
        exit(EXIT_FAILURE);
    }
}

void check_CUDA_error(const char *msg) {
    cudaError_t err = cudaGetLastError();
    if(cudaSuccess != err) {
        fprintf(stderr, "ERRORE CUDA: >%s<: >%s<. Eseguo: EXIT\n", msg, cudaGetErrorString(err) );
        exit(-1);
    }
}

In [68]:
%%file src/generate_n_b_couples.cpp

#include "../include/generate_n_b_couples.hpp"
#include "../include/macros.hpp"

// c library
// for srand() and rand()
#include <time.h>
#include <stdlib.h>

// c++ library for file stream
#include <fstream>

std::vector<std::pair<int, int>> generate_list_of_all_n_b(int min_input_size, int max_input_size, int max_num_of_b_per_n, double to_multiply, int to_sum, int min_blocking_factor, int max_num_tests, int seed) {

    // initialize seed
    srand(seed);

    std::vector<std::pair<int, int>> list_of_all_n_b;

    for (int n = min_input_size; n <= max_input_size; n = mmax(((int) (to_multiply * (double) n)) + to_sum, (n+1)) ) {

        // use mmax 5 different blocking factors
        int B_used[max_num_of_b_per_n];
        // initially all are -1
        for (int i = 0; i < max_num_of_b_per_n; i++) B_used[i] = -1;

        // index of the currently used B 
        int cur_B_idx = -1;

        // generate randomly B check if it is a divisor of n and not already used.
        // generate maximum max_num_tests random B (necessary to avoid non-termination, maybe n is prime)
        for (int tests = 0; tests < max_num_tests && cur_B_idx < max_num_of_b_per_n; tests++) {

            // range for b is between 0 and n/2
            int B = rand() % mmin(n/2, MAX_BLOCKING_FACTOR);
            // but if it is zero then use B=n
            B = (B == 0) ? n : B;

            // test if it is ok to be executed (b is a new divisor)
            bool is_ok = (n % B == 0) && (B <= MAX_BLOCKING_FACTOR) && (B >= min_blocking_factor); 
            for (int i = 0; (i <= cur_B_idx) && is_ok; i++) is_ok = (B != B_used[i]);

            if (is_ok) {
                B_used[++cur_B_idx] = B;
                list_of_all_n_b.push_back(std::make_pair(n, B));
                //printf("n: %d, b: %d\n", n, B);
            }
        }
    }

    return list_of_all_n_b;
}

void print_list_to_file(std::vector<std::pair<int, int>> list_of_all_n_b, std::string filename) {

    std::ofstream myfile;
    myfile.open(filename);
    myfile << "n,b\n";
        
    for (int i = 0; i < list_of_all_n_b.size(); i++) {

        int n = list_of_all_n_b[i].first;
        int b = list_of_all_n_b[i].second;
        myfile << n << "," << b << "\n";
    }
    myfile.close();
}



In [69]:
%%file src/handle_arguments_and_execute.cu

#include "../include/adj_matrix_utils.cuh"
#include "../include/adj_matrix_utils.hpp"
#include "../include/cuda_errors_utils.cuh"
#include "../include/host_floyd_warshall.hpp"
#include "../include/macros.hpp"
#include "../include/performance_test.cuh"
#include "../include/statistical_test.hpp"

// c libraries
// for printing
#include <stdio.h>
// to define a seed using current time with time(NULL)
#include <time.h>
// for pseudo-random generation with srand() for seed initialization and rand() for pseudo-random number
#include <stdlib.h>

// c++ libraries
// for argv handling as vector of string
#include <string>
#include <vector>

int handle_arguments_and_execute(int argc, char *argv[], void (*f) (int* arr_matrix, int n, int b)) {

    std::vector<std::string> str_args(argv, argv + argc);

    std::string exec_option;
    if (argc > 1) exec_option = str_args[1];

    if (argc == 1 || exec_option == "--help") {
        printf("Usage: %s <exec_option> [-n=<n>, -b=<b>, -t=<t> [-s=<s>]]:\n", argv[0]);
        printf(" where <exec_option>=test for statistical testing or <exec_option>=perf for nvprof profiling\n");
        printf("If <exec_option>=perf then specify n (matrix dimension), b (blocking factor), t (number of tests), [ s (seed), by default is random ]\n");
        return 1;
    }

    if (exec_option == "test") {

        MultiSizeTestParameters my_params;
        my_params.f = f;
        my_params.g = &host_array_floyd_warshall_blocked;
        my_params.start_input_size = 30;
        my_params.end_input_size = 150;
        my_params.to_multiply = RANDOM_CONSTANT;
        my_params.to_sum      = RANDOM_CONSTANT;
        my_params.min_blocking_factor = 2;

        print_multi_size_test_parameters(my_params);
        multi_size_statistical_test(my_params);
        

    } else if (exec_option == "perf") {
        
        int n = -1, b = -1, t = -1, s = -1;
        if (str_args.size() < 5) {
            printf("Missing n,t,b parameters\n");
            return 2;
        }
        for (int i = 2; i < argc; i++) {
            if(str_args[i].size() < 4) {
                //mmin size for n,b,t parameters is 4 (ex. -n=5)
                printf("Uncorrect syntax parameter, use -<param>=<value>\n");
                return 3;
            }
            if(str_args[i][0] == '-' && str_args[i][2] == '=') {
                int val = std::stoi((str_args[i]).substr(3));
                if(     str_args[i][1] == 'n') n = val; // mandatory: matrix size
                else if(str_args[i][1] == 'b') b = val; // mandatory: blocking factor size
                else if(str_args[i][1] == 't') t = val; // mandatory: number of tests to execute
                else if(str_args[i][1] == 's') s = val; // optional:  seed
                else {
                    printf("Parameter not recognised\n");
                    return 4;
                }
            } 
        }
        if (n <= 0 || b <= 0 || t <= 0) {
            printf("n, b, t must all be specified and must be positive integers\n");
            return 5;
        }
        if ((b > n) || (n % b > 0)) {
            printf("b must be a divisor of n\n");
            return 6;
        }
        if (s == -1) s = time(NULL);
        printf("seed: %d\n", s);
        do_nvprof_performance_test(f, n, b, t, s);

    } else {
        printf("<exec_option>=%s not recognised, try run: %s --help\n", argv[1], argv[0]);
        return 7;
    }

    return 0;
}

In [70]:
%%file src/host_floyd_warshall.cpp

#include "../include/host_floyd_warshall.hpp"
#include "../include/adj_matrix_utils.hpp"

// ---------------------------------------------------------------------------
// Matrix data structure version

void host_matrix_floyd_warshall(int **matrix, int n) {
    for(int k = 0; k < n; k++) {
        for(int i = 0; i < n; i++) {
            for(int j = 0; j < n; j++) {
                int a = matrix[i][j];
                int b = sum_if_not_infinite(matrix[i][k], matrix[k][j], INF);
                matrix[i][j] = mmin(a, b);
            }
        }
    }
}

void host_matrix_floyd_warshall_blocked(int **matrix, int n, int B) {

    int num_rounds = n/B;

    for(int t = 0; t < num_rounds; t++) { 

        //host_matrix_execute_round(int **matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        host_matrix_execute_round(matrix, n, t, t, t, B);

        //phase 2 blocks left
        for (int j = t-1; j >= 0; j--) {
            host_matrix_execute_round(matrix, n, t, t, j, B);
        }

        //phase 2 blocks above
        for (int i = t-1; i >= 0; i--) {
            host_matrix_execute_round(matrix, n, t, i, t, B);
        }

        //phase 2 blocks below
        for (int i = t+1; i < num_rounds; i++) {
            host_matrix_execute_round(matrix, n, t, i, t, B);
        }

        //phase 2 blocks right
        for (int j = t+1; j < num_rounds; j++) {
            host_matrix_execute_round(matrix, n, t, t, j, B);
        }
        
        //phase 2,3: remaining blocks
        //phase 3 blocks above and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t-1; i >= 0; i--) {
                host_matrix_execute_round(matrix, n, t, i, j, B);
            }
        }
        //phase 3 blocks above and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t-1; i >= 0; i--) {
                host_matrix_execute_round(matrix, n, t, i, j, B);
            }
        }
        //phase 3 blocks below and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t+1; i < num_rounds; i++) {
                host_matrix_execute_round(matrix, n, t, i, j, B);
            }
        }      
        //phase 3 blocks below and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t+1; i < num_rounds; i++) {
                host_matrix_execute_round(matrix, n, t, i, j, B);
            }
        }   
        
    }
}

void host_matrix_execute_round(int **matrix, int n, int t, int row, int col, int B) {

    //foreach k: t*B <= t < t+B
    int block_start = t * B;
    int block_end = (t+1) * B;
    int row_start = row * B;
    int row_end = (row+1) * B;
    int col_start = col * B;
    int col_end = (col+1) * B;

    for (int k = block_start; k < block_end; k++) {
        //foreach i,j in the self-dependent block
        for (int i = row_start; i < row_end; i++) {
            for (int j = col_start; j < col_end; j++) {
                int a = matrix[i][j];
                int x1 = matrix[i][k];
                int x2 =  matrix[k][j];
                int b = sum_if_not_infinite(matrix[i][k], matrix[k][j], INF);
                matrix[i][j] = mmin(a, b);
                //print_matrix(matrix, n, n);
            }
        }
    }
}

// ---------------------------------------------------------------------------
// Array data structure version

void host_array_floyd_warshall(int *matrix, int n) {
    for(int k = 0; k < n; k++) {
        for(int i = 0; i < n; i++) {
            for(int j = 0; j < n; j++) {
                int a = matrix[i*n + j];
                int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF);
                matrix[i*n + j] = mmin(a, b);
            }
        }
    }
}

void host_array_floyd_warshall_blocked(int *matrix, int n, int B) {

    int num_rounds = n/B;

    for(int t = 0; t < num_rounds; t++) { 

        //host_matrix_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        host_array_execute_round(matrix, n, t, t, t, B);

        //phase 2 blocks left
        for (int j = t-1; j >= 0; j--) {
            host_array_execute_round(matrix, n, t, t, j, B);
        }

        //phase 2 blocks above
        for (int i = t-1; i >= 0; i--) {
            host_array_execute_round(matrix, n, t, i, t, B);
        }

        //phase 2 blocks below
        for (int i = t+1; i < num_rounds; i++) {
            host_array_execute_round(matrix, n, t, i, t, B);
        }

        //phase 2 blocks right
        for (int j = t+1; j < num_rounds; j++) {
            host_array_execute_round(matrix, n, t, t, j, B);
        }
        
        //phase 2,3: remaining blocks
        //phase 3 blocks above and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t-1; i >= 0; i--) {
                host_array_execute_round(matrix, n, t, i, j, B);
            }
        }
        //phase 3 blocks above and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t-1; i >= 0; i--) {
                host_array_execute_round(matrix, n, t, i, j, B);
            }
        }
        //phase 3 blocks below and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t+1; i < num_rounds; i++) {
                host_array_execute_round(matrix, n, t, i, j, B);
            }
        }      
        //phase 3 blocks below and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t+1; i < num_rounds; i++) {
                host_array_execute_round(matrix, n, t, i, j, B);
            }
        }   
        
    }
}

void host_array_execute_round(int *matrix, int n, int t, int row, int col, int B) {
    //foreach k: t*B <= t < t+B
    int block_start = t * B;
    int block_end = (t+1) * B;
    int row_start = row * B;
    int row_end = (row+1) * B;
    int col_start = col * B;
    int col_end = (col+1) * B;
    for (int k = block_start; k < block_end; k++) {
        //foreach i,j in the self-dependent block
        for (int i = row_start; i < row_end; i++) {
            for (int j = col_start; j < col_end; j++) {
                int a = matrix[i*n + j];
                int x1 = matrix[i*n + k];
                int x2 =  matrix[k*n + j];
                int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF);
                matrix[i*n + j] = mmin(a, b);
                //print_arr_matrix(matrix, n, n);
            }
        }
    }
}

In [71]:
%%file src/performance_test.cu


#include <stdio.h>
#include <stdlib.h>

#include <cuda_profiler_api.h>

#include "../include/adj_matrix_utils.hpp"
#include "../include/performance_test.cuh"

void do_nvprof_performance_test(void (*floyd_warshall_arr_algorithm)(int* matrix, int n, int B), int input_size, int blocking_factor, int number_of_tests, int seed) {

    int* arr_matrix = allocate_arr_matrix(input_size, input_size);

    for (int i=0; i<number_of_tests; i++) {

        populate_arr_adj_matrix(arr_matrix, input_size, seed*(i+1), false);

        cudaProfilerStart();
        floyd_warshall_arr_algorithm(arr_matrix, input_size, blocking_factor);
        cudaProfilerStop();

        printf("Performed test number %d\n", i);
    }
}

In [72]:
%%file src/statistical_test.cpp


#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cassert>
#include <cstring>

#include "../include/adj_matrix_utils.hpp"
#include "../include/generate_n_b_couples.hpp"
#include "../include/host_floyd_warshall.hpp"
#include "../include/statistical_test.hpp"

bool exec_single_single_statistical_test(ExecSingleSizeTestParameters params) {

    //just a rename of the same pointer
    int *f_input = params.input_instance;

    //define an input copy and allocate memory its memory
    int *g_input = allocate_arr_matrix(params.input_size, params.input_size);

    //make a copy of f_input to g_input
    memcpy((void*) g_input, (void*) f_input, params.input_size * params.input_size * sizeof(int));
    
    //classic floyd_warshall on host, used to compare output
    params.f(f_input, params.input_size, params.blocking_factor);

    //function to test execution
    params.g(g_input, params.input_size, params.blocking_factor);

    //return true <==> foreach 0 <= i,j < n : input[i,j] = test[i,j]
    return same_arr_matrixes(f_input, g_input, params.input_size, params.input_size, false);
}


int call_single_size_statistical_test(CallSingleSizeTestParameters params) {

    /*
    printf("Performing statistical test with:\n");
    printf("\t%d executions\n", n_tests);
    if (use_always_seed==RANDOM_SEED) {
        printf("\tseed=RANDOM\n");
    } else {
        printf("\tseed=%d\n", use_always_seed);
    }

    printf("\tinput_size=%d\n\tblocking_factor=%d\n\n", input_size, blocking_factor);
    */

    int n_wrong = 0;

    //matrix initialization
    int *input_instance = allocate_arr_matrix(params.input_size, params.input_size);

    int i = 0;
    for (; i < params.n_tests; i++)
    {
        // Progression status print
        if((i > 0) && (i % (params.n_tests / params.print_progress_perc) == 0)) {
            double perc = ((double) i) / ((double) params.n_tests);
            printf("%d%%: %d of %d\n", (int) (perc*100), i, params.n_tests);
        }
        
        // if necessary, generate (pseudo) random input instance
        params.seed = (params.seed == RANDOM_SEED) ? clock() : params.seed;
        
        populate_arr_adj_matrix(input_instance, params.input_size, params.seed, false);

        //define exec single test params
        ExecSingleSizeTestParameters exec_params;
        // most of parameters are copied as received:
        exec_params.f               = params.f;
        exec_params.g               = params.g;
        exec_params.input_size      = params.input_size;
        exec_params.blocking_factor = params.blocking_factor;
        // input instance is allocated and populated here (based on seed value)
        exec_params.input_instance  = input_instance; 

        // perform test
        if (!exec_single_single_statistical_test(exec_params)) {

            n_wrong++;

            if (params.print_failed_tests) printf("%d/%d)\tseed: %d --> ERROR!\n", i, params.n_tests, params.seed);
            
            if (params.stop_current_if_fail) break;
        }
    }

    free(input_instance);

    printf("Test ended. Performed %d/%d tests and got %d/%d errors\n", i, params.n_tests, n_wrong, params.n_tests);

    return n_wrong;
}

int multi_size_statistical_test(MultiSizeTestParameters params) {

    assert(params.end_input_size >= params.start_input_size);
    // stop_all_if_fail ==> stop_current_if_fail
    params.stop_current_if_fail = params.stop_current_if_fail || params.stop_all_if_fail;
    
    int seed = time(NULL);
    srand(seed);
    
    // calculate default values
    double rand_to_multiply = OBTAIN_VAL_TO_MULTIPLY(rand());
    int rand_to_sum = OBTAIN_VAL_TO_SUM(rand());
    // if parameter is random then use calculated values, else use parameter
    params.to_multiply = (params.to_multiply == RANDOM_CONSTANT) ? rand_to_multiply : params.to_multiply;
    params.to_sum =      (params.to_sum      == RANDOM_CONSTANT) ? rand_to_sum      : params.to_sum;
    // in case is used the parameter value, check if it is non-negative
    assert(params.to_multiply >= 0);
    assert(params.to_sum      >= 0);

    printf("Performing Multi-size statistical test:\n");
    printf("- Input sizes between %d and %d, increase is linear, using %f as costant multiplier\n", params.start_input_size, params.end_input_size, params.to_multiply);
    printf("- Blocking factor are generated randomly between [1, n/2] U {n}\n");
    printf("- Number of executions for each couple (n,B) used: %d\n\n", params.n_tests_per_round);

    int n_err_tot = 0;

    std::vector<std::pair<int, int>> list_of_all_n_b;
    list_of_all_n_b = generate_list_of_all_n_b(params.start_input_size, params.end_input_size, 5, params.to_multiply, params.to_sum, params.min_blocking_factor, 50, seed);

    for (int i = 0; i < list_of_all_n_b.size(); i++) {

        int n = list_of_all_n_b[i].first;
        int B = list_of_all_n_b[i].second;

        //printf("n: %d, B: %d\n", n, B);

        //define exec single test params
        CallSingleSizeTestParameters single_test_params;
        // most of parameters are copied as received:
        single_test_params.f                    = params.f;
        single_test_params.g                    = params.g;
        single_test_params.seed                 = params.seed;
        single_test_params.n_tests              = params.n_tests_per_round;
        single_test_params.print_progress_perc  = params.print_progress_perc;
        single_test_params.print_failed_tests   = params.print_failed_tests;
        single_test_params.stop_current_if_fail = params.stop_current_if_fail;
        // the couple (n,B) is calculated here
        single_test_params.input_size      = n;
        single_test_params.blocking_factor = B;

        //execute test
        int n_err = call_single_size_statistical_test(single_test_params);
        
        // count errors
        n_err_tot += n_err;
        if (n_err > 0 && params.stop_all_if_fail) {
            return n_err_tot;
        }
        
        printf("Cumulative errors at size=%d, blocking_factor=%d: %d (%d new ones)\n\n", n, B, n_err_tot, n_err);
        
    }

    return n_err_tot;
}

void print_multi_size_test_parameters(MultiSizeTestParameters params) {
    printf("MultiSizeTestParameters:\n");
    printf("- pointer to test func:\t%p\n", &params.f);
    printf("- pointer to comp func:\t%p\n", &params.g);
    printf("- start input size:\t%d\n", params.start_input_size);
    printf("- end input size:\t%d\n", params.end_input_size);
    printf("- costant multiplier:\t");
    if (params.seed == RANDOM_CONSTANT) printf("RANDOM\n");
    else                                printf("%f\n", params.to_multiply);
    printf("- seed:\t\t\t");
    if (params.seed == RANDOM_SEED)     printf("RANDOM\n");
    else                                printf("%d\n", params.seed);
    printf("- n tests per round:\t%d\n", params.n_tests_per_round);
    printf("- print progress perc:\t%d%%\n", (100 / params.print_progress_perc));
    printf("- stop current if fail:\t%s\n", bool_to_string(params.stop_current_if_fail));
    printf("- stop all if fail:\t%s\n", bool_to_string(params.stop_all_if_fail));
    printf("- print failed tests:\t%s\n", bool_to_string(params.print_failed_tests));
    printf("- blocking factor:\tRANDOM\n");
    printf("\n");
}

void print_call_single_size_statistical_test_parameters(CallSingleSizeTestParameters params) {
    printf("CallSingleSizeTestParameters:\n");
    printf("- pointer to test func:\t%p\n", &params.f);
    printf("- pointer to comp func:\t%p\n", &params.g);
    printf("- input size:\t%d\n", params.input_size);
    printf("- blocking factor:\t%d\n", params.blocking_factor);
    printf("- seed:\t\t\t");
    if (params.seed == RANDOM_SEED)     printf("RANDOM\n");
    else                                printf("%d\n", params.seed);
    printf("- n tests:\t\t%d\n", params.n_tests);
    printf("- print progress perc:\t%d%%\n", (100 / params.print_progress_perc));
    printf("- stop current if fail:\t%s\n", bool_to_string(params.stop_current_if_fail));
    printf("- print failed tests:\t%s\n", bool_to_string(params.print_failed_tests));
    printf("\n");
}

void print_exec_single_size_statistical_test_parameters(ExecSingleSizeTestParameters params) {
    printf("ExecSingleSizeTestParameters:\n");
    printf("- pointer to test func:\t%p\n", &params.f);
    printf("- pointer to comp func:\t%p\n", &params.g);
    printf("- input size:\t%d\n", params.input_size);
    printf("- blocking factor:\t%d\n", params.blocking_factor);
    printf("- pointer to input data:\t%p\n", &params.input_instance);
    printf("\n");
}



In [73]:
# END SRC REGION
# ----------------------------------------------------------------------------------------------------------------------------------------
# START MAIN REGION

In [74]:
%%file device_floyd_warshall_v_1_0_ERROR.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_0(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_0(int *matrix, int n, int t, int row, int col, int B);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_0);

}

__global__ void execute_round_device_v_1_0(int *matrix, int n, int t, int row, int col, int B) {

    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    int i = tid/n;  // row
    int j = tid%n;  // col

    //foreach k: t*B <= t < t+B
    for (int k = t * B; k < (t+1) * B; k++) {

        int a, b;
        bool run_this = ((i >= row*B) && (i < (row+1)*B) && (j >= col*B) && (j < (col+1)*B));

        // check if thread correspond to one of the cells in current block
        if (run_this) {

            // WARNING: do NOT put the macro directly into 
            a = matrix[i*n + j];
            b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 
        }

        __syncthreads();


        if (run_this) {
            matrix[i*n + j] = mmin(a, b);
        }
        
        __syncthreads();

    }
}


void floyd_warshall_blocked_device_v_1_0(int *matrix, int n, int B) {

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));
    
    int num_rounds = n/B;

    int num_blocks = num_rounds*num_rounds;
    int thread_per_block = B*B; 
    

    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, t, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        //phase 2 blocks left
        for (int j = t-1; j >= 0; j--) {
            execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, t, j, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks above
        for (int i = t-1; i >= 0; i--) {
            execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, t, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks below
        for (int i = t+1; i < num_rounds; i++) {
            execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, t, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks right
        for (int j = t+1; j < num_rounds; j++) {
            execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, t, j, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        HANDLE_ERROR(cudaDeviceSynchronize());
        
        //phase 3 blocks above and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t-1; i >= 0; i--) {
                execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks above and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t-1; i >= 0; i--) {
                execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks below and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t+1; i < num_rounds; i++) {
                execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }      
        //phase 3 blocks below and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t+1; i < num_rounds; i++) {
                execute_round_device_v_1_0<<<num_blocks, thread_per_block>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }

        // HANDLE_ERROR(cudaDeviceSynchronize());   
    }

    HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));

    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}



In [75]:
%%file device_floyd_warshall_v_1_1_pitch.cu

#include "include/include_needed_libraries.cuh"


//main device code
void floyd_warshall_blocked_device_v_1_1_pitch(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_1_pitch(int *matrix, int n, int t, int row, int col, int B, size_t pitch);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_1_pitch);

}


void floyd_warshall_blocked_device_v_1_1_pitch(int *matrix, int n, int B) {
    
    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    size_t pitch;                          //size in bytes of memory allocated to guarantee alignment
    size_t width = n * sizeof(int);
    size_t height = n;

    //cudaMallocPitch(&devPtr, &devPitch, N_cols * sizeof(type), N_rows);

    HANDLE_ERROR(cudaMallocPitch( (void**) &dev_rand_matrix, &pitch, width, height));
    //HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n * n * sizeof(int), cudaMemcpyHostToDevice));
    HANDLE_ERROR(cudaMemcpy2D(dev_rand_matrix, pitch, matrix, width, width, height, cudaMemcpyHostToDevice));


    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        // printf("start self dependent, t:%d, row:%d, col:%d\n",t,t,t);
        execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, t, B, pitch);
        HANDLE_ERROR(cudaDeviceSynchronize());

        //phase 2 blocks left
        for (int j = t-1; j >= 0; j--) {
            // printf("start phase 2 blocks left, t:%d, row:%d, col:%d\n",t,t,j);
            execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, j, B, pitch);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks above
        for (int i = t-1; i >= 0; i--) {
            // printf("start phase 2 blocks above, t:%d, row:%d, col:%d\n",t,i,t);
            execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, t, B, pitch);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks below
        for (int i = t+1; i < num_rounds; i++) {
            // printf("start phase 2 blocks below, t:%d, row:%d, col:%d\n",t,i,t);
            execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, t, B, pitch);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks right
        for (int j = t+1; j < num_rounds; j++) {
            // printf("start phase 2 blocks right, t:%d, row:%d, col:%d\n",t,t,j);
            execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, j, B, pitch);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        HANDLE_ERROR(cudaDeviceSynchronize());
        
        //phase 3 blocks above and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t-1; i >= 0; i--) {
                // printf("start phase 3 blocks above and right, t:%d, row:%d, col:%d\n",t,i,j);
                execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B, pitch);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks above and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t-1; i >= 0; i--) {
                // printf("start phase 3 blocks above and left, t:%d, row:%d, col:%d\n",t,i,j);
                execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B, pitch);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks below and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t+1; i < num_rounds; i++) {
                // printf("start phase 3 blocks below and left, t:%d, row:%d, col:%d\n",t,i,j);
                execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B, pitch);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }      
        //phase 3 blocks below and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t+1; i < num_rounds; i++) {
                // printf("start phase 3 blocks below and right, t:%d, row:%d, col:%d\n",t,i,j);
                execute_round_device_v_1_1_pitch<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B, pitch);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }

        HANDLE_ERROR(cudaDeviceSynchronize());  
    }

    // HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaMemcpy2D(matrix, width, dev_rand_matrix, pitch, width, height, cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}

__global__ void execute_round_device_v_1_1_pitch(int *matrix, int n, int t, int row, int col, int B, size_t pitch) {

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;
    
    int i = tid_x + row * B;  // row
    int j = tid_y + col * B;  // col
    
    int *cell_i_j = pitched_pointer(matrix, i, j, pitch); //(int *)((char*) matrix + i * pitch) + j;
    //int cell_i_j_bef = *cell_i_j;

    /*
    printf(
        "tid_x:%d, tid_y:%d, i:%d, j:%d, threadIdx.x:%d, blockIdx.x:%d, blockDim.x:%d, threadIdx.y:%d, blockIdx.y:%d, blockDim.y:%d\n",
        tid_x, tid_y, i, j, threadIdx.x, blockIdx.x, blockDim.x, threadIdx.y, blockIdx.y, blockDim.y
    );
    */

    //foreach k: t*B <= t < t+B
    for (int k = t * B; k < (t+1) * B; k++) {

        int* cell_k_j = pitched_pointer(matrix, k, j, pitch); //(int *)((char*) matrix + k * pitch) + j;
        int* cell_i_k = pitched_pointer(matrix, i, k, pitch); //(int *)((char*) matrix + i * pitch) + k;

        int using_k_path = sum_if_not_infinite(*cell_i_k, *cell_k_j, INF); 

        if (using_k_path < *cell_i_j) {
            *cell_i_j = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d, max_k:%d, ik:%02d, kj:%02d, ij_bef:%02d, ij_aft:%02d\n", i, j, k, (t+1)*B, (mmin(*cell_i_k, 99)), (mmin(*cell_k_j, 99)), (mmin(cell_i_j_bef, 99)), (mmin(*cell_i_j, 99)));
        // if (tid_x==0 && tid_y==0) printf("i:%d, j:%d, k:%d, max_k:%d\n", i, j, k, (t+1)*B);      
        
        __syncthreads();


        /*
        if((i % 2 == 0) && (j % 2 == 0)) {
            printf("k:%d\n",k);
            print_matrix_device(matrix, n, n, pitch);
            printf("\n");
        }
        */

    }
    
}


In [76]:
%%file device_floyd_warshall_v_1_1.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_1(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_1(int *matrix, int n, int t, int row, int col, int B);


int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_1);

}

__global__ void execute_round_device_v_1_1(int *matrix, int n, int t, int row, int col, int B) {

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    int i = tid_x + row * B;  // row
    int j = tid_y + col * B;  // col
    
    /*
    printf(
        "tid_x:%d, tid_y:%d, i:%d, j:%d, threadIdx.x:%d, blockIdx.x:%d, blockDim.x:%d, threadIdx.y:%d, blockIdx.y:%d, blockDim.y:%d\n",
        tid_x, tid_y, i, j, threadIdx.x, blockIdx.x, blockDim.x, threadIdx.y, blockIdx.y, blockDim.y
    );
    */

    //foreach k: t*B <= t < t+B
    for (int k = t * B; k < (t+1) * B; k++) {

        int a, b;
        bool run_this = true; // i>=row * B && i<(row+1) * B && j>=col * B && j<(col+1) * B;

        // check if thread correspond to one of the cells in current block
        if (run_this) {

            // WARNING: do NOT put the macro directly into 
            a = matrix[i*n + j];
            b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 
        }

        // __syncthreads();

        if (b < a) {
            matrix[i*n + j] = b;
        }
        
        __syncthreads();

        /*
        if((i % 2 == 0) && (j % 2 == 0)) {
            printf("k:%d\n",k);
            //print_matrix_device(matrix, n, n);
            printf("\n");
        }
        */
    }
}

void floyd_warshall_blocked_device_v_1_1(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        //phase 2 blocks left
        for (int j = t-1; j >= 0; j--) {
            execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, j, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks above
        for (int i = t-1; i >= 0; i--) {
            execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, t, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks below
        for (int i = t+1; i < num_rounds; i++) {
            execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, t, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        //phase 2 blocks right
        for (int j = t+1; j < num_rounds; j++) {
            execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, t, j, B);
            // HANDLE_ERROR(cudaDeviceSynchronize());  
        }

        HANDLE_ERROR(cudaDeviceSynchronize());
        
        //phase 3 blocks above and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t-1; i >= 0; i--) {
                execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks above and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t-1; i >= 0; i--) {
                execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }
        //phase 3 blocks below and left
        for (int j = t-1; j >= 0; j--) {
            for (int i = t+1; i < num_rounds; i++) {
                execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }      
        //phase 3 blocks below and right
        for (int j = t+1; j < num_rounds; j++) {
            for (int i = t+1; i < num_rounds; i++) {
                execute_round_device_v_1_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, i, j, B);
                // HANDLE_ERROR(cudaDeviceSynchronize());  
            }
        }

        HANDLE_ERROR(cudaDeviceSynchronize());   
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));

    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


In [77]:
%%file device_floyd_warshall_v_1_2.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_2(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_2_phase_2(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_2_phase_3(int *matrix, int n, int t, int B);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_2);

}


void floyd_warshall_blocked_device_v_1_2(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_1_2_phase_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        // Phase 2/3 thread matrix is made by n*n threads, divided in num_rounds*num_rounds blocks
        dim3 num_blocks_phase_2_3(num_rounds, num_rounds);  

        execute_round_device_v_1_2_phase_2<<<num_blocks_phase_2_3, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        execute_round_device_v_1_2_phase_3<<<num_blocks_phase_2_3, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}

__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    int i = tid_x + t * B;  // row
    int j = tid_y + t * B;  // col

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (b < matrix[i*n + j]) {
            matrix[i*n + j] = b;
        }
        
        __syncthreads();
    }
}

__global__ void execute_round_device_v_1_2_phase_2(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix 
    // ("-" and "." blocks are just kept inactive using IF statement)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .


    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        if (
            /* row index is contained in s.d. block and column index is outside */
            ( BLOCK_START(t,B)<=i<BLOCK_END(t,B) && (j<BLOCK_START(t,B) || j>=BLOCK_END(t,B)) )   ||  

            /* column index is contained in s.d. block and row index is outside */
            ( BLOCK_START(t,B)<=j<BLOCK_END(t,B) && (i<BLOCK_START(t,B) || i>=BLOCK_END(t,B)) ) 
            ) {

            int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

            if (b < matrix[i*n + j]) {
                matrix[i*n + j] = b;
            }
        }

        __syncthreads();
    }
}

__global__ void execute_round_device_v_1_2_phase_3(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix 
    // ("-" blocks are just kept inactive using IF statement)

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        if (
            /* above and right or left */
            ( i>=BLOCK_END(t,B) && (j<BLOCK_START(t,B) || j>=BLOCK_END(t,B)) )   ||  

            /* under and right or left */
            ( i<BLOCK_START(t,B) && (j<BLOCK_START(t,B) || j>=BLOCK_END(t,B)) ) 
            ) {

            int using_k_path = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

            if (using_k_path < matrix[i*n + j]) {
                matrix[i*n + j] = using_k_path;
            }
        }

        __syncthreads();
    }
}



In [120]:
%%file device_floyd_warshall_v_1_3_pitch.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_3_pitch(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B, size_t pitch);
__global__ void execute_round_device_v_1_3_phase_2(int *matrix, int n, int t, int B, size_t pitch);
__global__ void execute_round_device_v_1_3_phase_3(int *matrix, int n, int t, int B, size_t pitch);

int main(int argc, char *argv[]) {

    //do_nvprof_performance_test(&floyd_warshall_blocked_device_v_1_3_pitch, 50, 10, 1, time(NULL));
    int n = 10;
    int b = 2;
    int* matrix = allocate_arr_matrix(n, n);
    populate_arr_adj_matrix(matrix, n, time(NULL), true);
    floyd_warshall_blocked_device_v_1_3_pitch(matrix, n, b);
    return 0;

    //return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_3_pitch);

}

void floyd_warshall_blocked_device_v_1_3_pitch(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    size_t pitch;                          //size in bytes of memory allocated to guarantee alignment
    size_t width = n * sizeof(int);
    size_t height = n;

    //cudaMallocPitch(&devPtr, &devPitch, N_cols * sizeof(type), N_rows);

    HANDLE_ERROR(cudaMallocPitch( (void**) &dev_rand_matrix, &pitch, width, height));
    //HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n * n * sizeof(int), cudaMemcpyHostToDevice));
    HANDLE_ERROR(cudaMemcpy2D(dev_rand_matrix, pitch, matrix, width, width, height, cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        printf("round %d of %d\n", t, num_rounds);
        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        printf("1 start\n");

        execute_round_device_v_1_2_phase_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B, pitch);
        HANDLE_ERROR(cudaDeviceSynchronize());
        
        printf("1 end\n");

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t


        dim3 num_blocks_phase_2(2, num_rounds-1);  

        printf("2 start\n");

        execute_round_device_v_1_3_phase_2<<<num_blocks_phase_2, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B, pitch);
        HANDLE_ERROR(cudaDeviceSynchronize());
        
        printf("2 end\n");

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        printf("3 start\n");

        execute_round_device_v_1_3_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B, pitch);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
        
        printf("3 end\n");
    }

    // HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaMemcpy2D(matrix, width, dev_rand_matrix, pitch, width, height, cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B, size_t pitch) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    int i = tid_x + t * B;  // row
    int j = tid_y + t * B;  // col

    int *cell_i_j = pitched_pointer(matrix, i, j, pitch);

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int* cell_k_j = pitched_pointer(matrix, k, j, pitch); //(int *)((char*) matrix + k * pitch) + j;
        int* cell_i_k = pitched_pointer(matrix, i, k, pitch); //(int *)((char*) matrix + i * pitch) + k;
    
        int using_k_path = sum_if_not_infinite(*cell_i_k, *cell_k_j, INF); 

        if (using_k_path < *cell_i_j) {
            *cell_i_j = using_k_path;
        }
        
        __syncthreads();
    }
}

__global__ void execute_round_device_v_1_3_phase_2(int *matrix, int n, int t, int B, size_t pitch) {

    // Launched blocks and correspondent position in the matrix
    //  -   blockIdx.x says if I am iterating row or cols, 
    //  -   blockIdx.y says something about which row or col)
    //  -   threadIdx.x and threadIdx.y are relative position of cell in block

    //  L1  L2  L3  R1  R2
    //  U1  U2  U3  D1  D2

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    int i, j;

    if (blockIdx.x == 0) {  

        // it's a row ...
        i = BLOCK_START(t, B) + threadIdx.x;

        if (blockIdx.y < t) {

            // ... and it's the left one
            j = BLOCK_START(blockIdx.y, B) + threadIdx.y;

        } else {
            
            // ... and it's the right one
            j = BLOCK_START(blockIdx.y, B) + B + threadIdx.y;
        }
    } else {

        // it's a column ...
        j = BLOCK_START(t, B) + threadIdx.y;

        if (blockIdx.y < t) {

            // ... and it's the up one
            i = BLOCK_START(blockIdx.y, B) + threadIdx.x;

        } else {

            // ... and it's the down one
            i = BLOCK_START(blockIdx.y, B) + B + threadIdx.x;
        }
    }

    int *cell_i_j = pitched_pointer(matrix, i, j, pitch); 

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        if (
            /* row index is contained in s.d. block and column index is outside */
            ( BLOCK_START(t,B)<=i<BLOCK_END(t,B) && (j<BLOCK_START(t,B) || j>=BLOCK_END(t,B)) )   ||  

            /* column index is contained in s.d. block and row index is outside */
            ( BLOCK_START(t,B)<=j<BLOCK_END(t,B) && (i<BLOCK_START(t,B) || i>=BLOCK_END(t,B)) ) 
            ) {

            int* cell_k_j = pitched_pointer(matrix, k, j, pitch); //(int *)((char*) matrix + k * pitch) + j;
            int* cell_i_k = pitched_pointer(matrix, i, k, pitch); //(int *)((char*) matrix + i * pitch) + k;
    
            int using_k_path = sum_if_not_infinite(*cell_i_k, *cell_k_j, INF); 
    
            if (using_k_path < *cell_i_j) {
                *cell_i_j = using_k_path;
            }
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();

    }
}


__global__ void execute_round_device_v_1_3_phase_3(int *matrix, int n, int t, int B, size_t pitch) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
           
    // if a thread is under t, add B as row offset to get right position in matrix
    if (blockIdx.x >= t)    i += B; 

    // if a thread is ar right of t, add B as col offset to get right position in matrix
    if (blockIdx.y >= t)    j += B;

    int *cell_i_j = pitched_pointer(matrix, i, j, pitch); 

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int* cell_k_j = pitched_pointer(matrix, k, j, pitch); //(int *)((char*) matrix + k * pitch) + j;
        int* cell_i_k = pitched_pointer(matrix, i, k, pitch); //(int *)((char*) matrix + i * pitch) + k;

        int using_k_path = sum_if_not_infinite(*cell_i_k, *cell_k_j, INF); 

        if (using_k_path < *cell_i_j) {
            *cell_i_j = using_k_path;
        }

        __syncthreads();
    }
}



In [79]:
%%file device_floyd_warshall_v_1_3.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_3(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_3_phase_2(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_3_phase_3(int *matrix, int n, int t, int B);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_3);

}

void floyd_warshall_blocked_device_v_1_3(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_1_2_phase_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        dim3 num_blocks_phase_2(2, num_rounds-1);  

        execute_round_device_v_1_3_phase_2<<<num_blocks_phase_2, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        execute_round_device_v_1_3_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_1_2_phase_1(int *matrix, int n, int t, int B) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    int i = tid_x + t * B;  // row
    int j = tid_y + t * B;  // col

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (b < matrix[i*n + j]) {
            matrix[i*n + j] = b;
        }
        
        __syncthreads();
    }
}

__global__ void execute_round_device_v_1_3_phase_2(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix
    //  -   blockIdx.x says if I am iterating row or cols, 
    //  -   blockIdx.y says something about which row or col)
    //  -   threadIdx.x and threadIdx.y are relative position of cell in block

    //  L1  L2  L3  R1  R2
    //  U1  U2  U3  D1  D2

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    int i, j;

    if (blockIdx.x == 0) {  

        // it's a row ...
        i = BLOCK_START(t, B) + threadIdx.x;

        if (blockIdx.y < t) {

            // ... and it's the left one
            j = BLOCK_START(blockIdx.y, B) + threadIdx.y;

        } else {
            
            // ... and it's the right one
            j = BLOCK_START(blockIdx.y, B) + B + threadIdx.y;
        }
    } else {

        // it's a column ...
        j = BLOCK_START(t, B) + threadIdx.y;

        if (blockIdx.y < t) {

            // ... and it's the up one
            i = BLOCK_START(blockIdx.y, B) + threadIdx.x;

        } else {

            // ... and it's the down one
            i = BLOCK_START(blockIdx.y, B) + B + threadIdx.x;
        }
    }

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        if (
            /* row index is contained in s.d. block and column index is outside */
            ( BLOCK_START(t,B)<=i<BLOCK_END(t,B) && (j<BLOCK_START(t,B) || j>=BLOCK_END(t,B)) )   ||  

            /* column index is contained in s.d. block and row index is outside */
            ( BLOCK_START(t,B)<=j<BLOCK_END(t,B) && (i<BLOCK_START(t,B) || i>=BLOCK_END(t,B)) ) 
            ) {

            int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

            if (b < matrix[i*n + j]) {
                matrix[i*n + j] = b;
            }
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();

    }
}


__global__ void execute_round_device_v_1_3_phase_3(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
           
    // if a thread is under t, add B as row offset to get right position in matrix
    if (blockIdx.x >= t)    i += B; 

    // if a thread is ar right of t, add B as col offset to get right position in matrix
    if (blockIdx.y >= t)    j += B;

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int using_k_path = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (using_k_path < matrix[i*n + j]) {
            matrix[i*n + j] = using_k_path;
        }

        __syncthreads();
    }
}



In [80]:
%%file device_floyd_warshall_v_1_4.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_1_4(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_1_4_phase_1(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_4_phase_2_row(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_4_phase_2_col(int *matrix, int n, int t, int B);
__global__ void execute_round_device_v_1_4_phase_3(int *matrix, int n, int t, int B);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_1_4);

}

void floyd_warshall_blocked_device_v_1_4(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_1_4_phase_1<<<num_blocks_phase_1, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        dim3 num_blocks_phase_2(1, num_rounds-1);  

        execute_round_device_v_1_4_phase_2_row<<<num_blocks_phase_2, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        execute_round_device_v_1_4_phase_2_col<<<num_blocks_phase_2, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);

        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        execute_round_device_v_1_4_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1>>>(dev_rand_matrix, n, t, B);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_1_4_phase_1(int *matrix, int n, int t, int B) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    int tid_x = threadIdx.x + blockIdx.x * blockDim.x;
    int tid_y = threadIdx.y + blockIdx.y * blockDim.y;

    int i = tid_x + t * B;  // row
    int j = tid_y + t * B;  // col

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (b < matrix[i*n + j]) {
            matrix[i*n + j] = b;
        }
        
        __syncthreads();
    }
}

__global__ void execute_round_device_v_1_4_phase_2_row(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix
    //  -   blockIdx.x says if I am iterating row or cols, 
    //  -   blockIdx.y says something about which row or col)
    //  -   threadIdx.x and threadIdx.y are relative position of cell in block

    //  L1  L2  L3  R1  R2

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    int i, j;

    // it's a row ...
    i = BLOCK_START(t, B) + threadIdx.x;

    if (blockIdx.y < t) {

        // ... and it's the left one
        j = BLOCK_START(blockIdx.y, B) + threadIdx.y;

    } else {
        
        // ... and it's the right one
        j = BLOCK_START(blockIdx.y, B) + B + threadIdx.y;
    }

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (b < matrix[i*n + j]) {
            matrix[i*n + j] = b;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();

    }
}

__global__ void execute_round_device_v_1_4_phase_2_col(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix
    //  -   blockIdx.x says if I am iterating row or cols, 
    //  -   blockIdx.y says something about which row or col)
    //  -   threadIdx.x and threadIdx.y are relative position of cell in block

    //  U1  U2  U3  D1  D2

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    int i, j;

    // it's a column ...
    j = BLOCK_START(t, B) + threadIdx.y;

    if (blockIdx.y < t) {

        // ... and it's the up one
        i = BLOCK_START(blockIdx.y, B) + threadIdx.x;

    } else {

        // ... and it's the down one
        i = BLOCK_START(blockIdx.y, B) + B + threadIdx.x;
    }

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int b = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (b < matrix[i*n + j]) {
            matrix[i*n + j] = b;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }
}


__global__ void execute_round_device_v_1_4_phase_3(int *matrix, int n, int t, int B) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
           
    // if a thread is under t, add B as row offset to get right position in matrix
    if (blockIdx.x >= t)    i += B; 

    // if a thread is ar right of t, add B as col offset to get right position in matrix
    if (blockIdx.y >= t)    j += B;

    //foreach k: t*B <= t < t+B
    for (int k = BLOCK_START(t,B); k < BLOCK_END(t,B); k++) {

        int using_k_path = sum_if_not_infinite(matrix[i*n + k], matrix[k*n + j], INF); 

        if (using_k_path < matrix[i*n + j]) {
            matrix[i*n + j] = using_k_path;
        }

        __syncthreads();
    }
}



In [81]:
%%file device_floyd_warshall_v_2_0.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_2_0(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_2_0_phase_1(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_0_phase_2_row(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_0_phase_2_col(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_0_phase_3(int *matrix, int n, int t);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_2_0);

}

void floyd_warshall_blocked_device_v_2_0(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_2_0_phase_1<<<num_blocks_phase_1, threads_per_block_phase_1, B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        // dim3 num_blocks_phase_2(1, num_rounds-1);  

        execute_round_device_v_2_0_phase_2_row<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        execute_round_device_v_2_0_phase_2_col<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);


        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        execute_round_device_v_2_0_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_2_0_phase_1(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    extern __shared__ int block_t_t_shared[];

    int i = threadIdx.x + t * blockDim.x;  // row abs index
    int j = threadIdx.y + t * blockDim.x;  // col abs index

    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[threadIdx.x*blockDim.x + k], 
            block_t_t_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }
        
        __syncthreads();
    }

    matrix[i*n + j] = block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y];
}

__global__ void execute_round_device_v_2_0_phase_2_row(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  L1  L2  L3  R1  R2      
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];
    
    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[(blockDim.x * blockDim.x)];

    // it's a row ...

    // abs row index 
    int i = BLOCK_START(t, blockDim.x) + threadIdx.x;    
    // abs col index   
    int j = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.y + ((blockIdx.x >= t) ? blockDim.x : 0); 

    // the block where I am working
    block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    // the self-dependent block already calculated in this round
    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        // Because we are doing rows:
        // -    matrix[i,abs_k] is in block_t_t_shared[threadIdx.x,k]
        // -    matrix[abs_k,j] is in block_i_j_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[threadIdx.x*blockDim.x + k], 
            block_i_j_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y];
}

__global__ void execute_round_device_v_2_0_phase_2_col(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  U1  U2  U3  D1  D2
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];

    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[blockDim.x*blockDim.x];

    // it's a column ...

    // abs row index 
    int i = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index 
    int j = BLOCK_START(t, blockDim.x) + threadIdx.y;

    // the block where I am working
    block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    // the self-dependent block already calculated in this round
    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {
        
        // Because we are doing columns:
        // -    matrix[i,k] is in block_i_j_shared[threadIdx.x,k]
        // -    matrix[k,j] is in block_t_t_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_i_j_shared[threadIdx.x*blockDim.x + k], 
            block_t_t_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y];
}


__global__ void execute_round_device_v_2_0_phase_3(int *matrix, int n, int t) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    extern __shared__ int shared_mem[];

    int* block_i_t_shared = &shared_mem[0];
    int* block_t_j_shared = &shared_mem[blockDim.x*blockDim.x];

    // abs row index
    int i = threadIdx.x + blockIdx.x * blockDim.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index
    int j = threadIdx.y + blockIdx.y * blockDim.y + ((blockIdx.y >= t) ? blockDim.x : 0);
    
    // since the cell i,j is read and written only by this thread
    // there is no need to copy its value to shared memory we can just us a local variable
    int cell_i_j = matrix[i*n + j];
        
    // In phase 3 I copy in two portions of my shared memory
    // the block corresponding to (t, this column) and (this row, t)
    block_i_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        i*n + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];
    block_t_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + j
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_i_t_shared[threadIdx.x*blockDim.x + k],
            block_t_j_shared[k*blockDim.x + threadIdx.y],
            INF
        ); 

        if (using_k_path < cell_i_j) {
            cell_i_j = using_k_path;
        }

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = cell_i_j;
}



In [82]:
%%file device_floyd_warshall_v_2_1_ERROR.cu

#include "include/include_needed_libraries.cuh"

//main device code
void floyd_warshall_blocked_device_v_2_1f(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_2_1f_phase_1(int *matrix, int n);
__global__ void execute_round_device_v_2_1f_phase_2_row(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_1f_phase_2_col(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_1f_phase_3(int *matrix, int n, int t);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_2_1f);

}

void floyd_warshall_blocked_device_v_2_1f(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n * n* sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;

    // phase 1: all the self-dependent blocks made in parallel before starting the rounds 
    // dim3 num_blocks_phase_1(1, 1);
    dim3 threads_per_block_phase_1(B, B);

    execute_round_device_v_2_1f_phase_1<<<num_rounds, threads_per_block_phase_1, B*B*sizeof(int)>>>(dev_rand_matrix, n);
    HANDLE_ERROR(cudaDeviceSynchronize());
     
    for(int t = 0; t < num_rounds; t++) { 
        

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        // dim3 num_blocks_phase_2(1, num_rounds-1);  

        execute_round_device_v_2_1f_phase_2_row<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        execute_round_device_v_2_1f_phase_2_col<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);


        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        execute_round_device_v_2_1f_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_2_1f_phase_1(int *matrix, int n) {

    // Launched block and correspondent position in the matrix

    //  t1  t2  t3  t4  t5  t6
    // (trasposed)

    //  t1  .   .   .   .   . 
    //  .   t2  .   .   .   . 
    //  .   .   t3  .   .   . 
    //  .   .   .   t4  .   .
    //  .   .   .   .   t5  . 
    //  .   .   .   .   .   t6

    extern __shared__ int block_t_t_shared[];

    int i = threadIdx.x + blockIdx.x * blockDim.x;  // row abs index
    int j = threadIdx.y + blockIdx.x * blockDim.x;  // col abs index

    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[threadIdx.x*blockDim.x + k], 
            block_t_t_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }
        
        __syncthreads();
    }

    matrix[i*n + j] = block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y];
}

__global__ void execute_round_device_v_2_1f_phase_2_row(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  L1  L2  L3  R1  R2      
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];
    
    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[(blockDim.x * blockDim.x)];

    // it's a row ...

    // abs row index 
    int i = BLOCK_START(t, blockDim.x) + threadIdx.x;    
    // abs col index   
    int j = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.y + ((blockIdx.x >= t) ? blockDim.x : 0); 

    // the block where I am working
    block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    // the self-dependent block already calculated in this round
    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        // Because we are doing rows:
        // -    matrix[i,abs_k] is in block_t_t_shared[threadIdx.x,k]
        // -    matrix[abs_k,j] is in block_i_j_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[threadIdx.x*blockDim.x + k], 
            block_i_j_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y];
}

__global__ void execute_round_device_v_2_1f_phase_2_col(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  U1  U2  U3  D1  D2
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];

    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[blockDim.x*blockDim.x];

    // it's a column ...

    // abs row index 
    int i = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index 
    int j = BLOCK_START(t, blockDim.x) + threadIdx.y;

    // the block where I am working
    block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[i*n + j];

    // the self-dependent block already calculated in this round
    block_t_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {
        
        // Because we are doing columns:
        // -    matrix[i,k] is in block_i_j_shared[threadIdx.x,k]
        // -    matrix[k,j] is in block_t_t_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_i_j_shared[threadIdx.x*blockDim.x + k], 
            block_t_t_shared[k*blockDim.x + threadIdx.y], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y]) {
            block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = block_i_j_shared[threadIdx.x*blockDim.x + threadIdx.y];
}


__global__ void execute_round_device_v_2_1f_phase_3(int *matrix, int n, int t) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    extern __shared__ int shared_mem[];

    int* block_i_t_shared = &shared_mem[0];
    int* block_t_j_shared = &shared_mem[blockDim.x*blockDim.x];

    // abs row index
    int i = threadIdx.x + blockIdx.x * blockDim.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index
    int j = threadIdx.y + blockIdx.y * blockDim.y + ((blockIdx.y >= t) ? blockDim.x : 0);
    
    // since the cell i,j is read and written only by this thread
    // there is no need to copy its value to shared memory we can just us a local variable
    int cell_i_j = matrix[i*n + j];
        
    // In phase 3 I copy in two portions of my shared memory
    // the block corresponding to (t, this column) and (this row, t)
    block_i_t_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        i*n + (BLOCK_START(t, blockDim.x) + threadIdx.y)
    ];
    block_t_j_shared[threadIdx.x*blockDim.x + threadIdx.y] = matrix[
        ((BLOCK_START(t, blockDim.x) + threadIdx.x) * n) + j
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_i_t_shared[threadIdx.x*blockDim.x + k],
            block_t_j_shared[k*blockDim.x + threadIdx.y],
            INF
        ); 

        if (using_k_path < cell_i_j) {
            cell_i_j = using_k_path;
        }

        __syncthreads();
    }

    // copy result in global memory
    matrix[i*n + j] = cell_i_j;
}



In [83]:
%%file device_floyd_warshall_v_2_2.cu

#include "include/include_needed_libraries.cuh"
#include "include/lcm.hpp"

#define ARR_MATRIX_INDEX(i,j,n) (i*n+j)
#define ARR_MATRIX_INDEX_TRASP(i,j,n) (i+n*j)

#define SHARED_BANK_N_INT 32
#define ARR_MATRIX_INDEX_BANK_CONFLICT(i, j, n, handle_bank_conflict) (i*n + j + (handle_bank_conflict ? i : 0))
#define ARR_MATRIX_SIZE_BANK_CONFICT(B,handle_bank_conflict) (B*B + (handle_bank_conflict ? (B-1) : 0))

//main device code
void floyd_warshall_blocked_device_v_2_2(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_2_2_phase_1(int *matrix, int n, int t, bool handle_bank_conflict);
__global__ void execute_round_device_v_2_2_phase_2_row(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_2_phase_2_col(int *matrix, int n, int t);
__global__ void execute_round_device_v_2_2_phase_3(int *matrix, int n, int t);

int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_2_2);
}

void floyd_warshall_blocked_device_v_2_2(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed mmax block size

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n*n*sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B;

    bool bank_conflict_phase_1 = lcm(SHARED_BANK_N_INT, B) <= (B-1)*B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_2_2_phase_1<<<
            num_blocks_phase_1, 
            threads_per_block_phase_1, 
            ARR_MATRIX_SIZE_BANK_CONFICT(B, bank_conflict_phase_1)*sizeof(int)
            >>>(dev_rand_matrix, n, t, bank_conflict_phase_1);

        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        // dim3 num_blocks_phase_2(1, num_rounds-1);  

        execute_round_device_v_2_2_phase_2_row<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        execute_round_device_v_2_2_phase_2_col<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);


        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 

        execute_round_device_v_2_2_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));
}


__global__ void execute_round_device_v_2_2_phase_1(int *matrix, int n, int t, bool handle_bank_conflict) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    extern __shared__ int block_t_t_shared[];

    int i = threadIdx.x + t * blockDim.x;  // row abs index
    int j = threadIdx.y + t * blockDim.x;  // col abs index

    block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, k, blockDim.x, handle_bank_conflict)], 
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(k, threadIdx.y, blockDim.x, handle_bank_conflict)], 
            INF
        ); 

        if (using_k_path < block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)]) {
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = using_k_path;
        }
        
        __syncthreads();
    }

    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)];
}

__global__ void execute_round_device_v_2_2_phase_2_row(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  L1  L2  L3  R1  R2      
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];
    
    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[(blockDim.x * blockDim.x)];

    // it's a row ...

    // abs row index 
    int i = BLOCK_START(t, blockDim.x) + threadIdx.x;    
    // abs col index   
    int j = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.y + ((blockIdx.x >= t) ? blockDim.x : 0); 

    // the block where I am working
    block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round (transposed to avoid bank conflict)
    block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];


    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        // Because we are doing rows:
        // -    matrix[i,abs_k] is in block_t_t_shared[threadIdx.x,k]
        // -    matrix[abs_k,j] is in block_i_j_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_i_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)];
}

__global__ void execute_round_device_v_2_2_phase_2_col(int *matrix, int n, int t) {

    // Launched block and correspondent position in the matrix

    //  U1  U2  U3  D1  D2
    //  (trasposed)

    //  .   .   .   U1  .   .
    //  .   .   .   U2  .   .
    //  .   .   .   U3  .   .
    //  L1  L2  L3  -   R1  R2
    //  .   .   .   D1  .   .
    //  .   .   .   D2  .   .

    extern __shared__ int shared_mem[];

    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[blockDim.x*blockDim.x];

    // it's a column ...

    // abs row index 
    int i = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index 
    int j = BLOCK_START(t, blockDim.x) + threadIdx.y;

    // the block where I am working (transposed to avoid bank conflict)
    block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round 
    block_t_t_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {
        
        // Because we are doing columns:
        // -    matrix[i,k] is in block_i_j_shared[threadIdx.x,k]
        // -    matrix[k,j] is in block_t_t_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_t_t_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)];
}


__global__ void execute_round_device_v_2_2_phase_3(int *matrix, int n, int t) {

    // Launched blocks and correspondent position in the matrix

    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  UL  UL  UL  UR  UR
    //  DL  DL  DL  DR  DR
    //  DL  DL  DL  DR  DR

    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR
    //  UL  UL  UL  -   UR  UR  
    //  -   -   -   -   -   - 
    //  DL  DL  DL  -   DR  DR
    //  DL  DL  DL  -   DR  DR

    extern __shared__ int shared_mem[];

    int* block_i_t_shared = &shared_mem[0];
    int* block_t_j_shared = &shared_mem[blockDim.x*blockDim.x];

    // abs row index
    int i = threadIdx.x + blockIdx.x * blockDim.x + ((blockIdx.x >= t) ? blockDim.x : 0);
    // abs col index
    int j = threadIdx.y + blockIdx.y * blockDim.y + ((blockIdx.y >= t) ? blockDim.x : 0);
    
    // since the cell i,j is read and written only by this thread
    // there is no need to copy its value to shared memory we can just us a local variable
    int cell_i_j = matrix[ARR_MATRIX_INDEX(i, j, n)];
        
    // In phase 3 I copy in two portions of my shared memory
    // the block corresponding to (t, this column) and (this row, t). 

    // (this row, t) is transposed to prevent bank conflict

    block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(i, (BLOCK_START(t, blockDim.x) + threadIdx.y), n)
    ];
    block_t_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX((BLOCK_START(t, blockDim.x) + threadIdx.x), j, n)
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)],
            block_t_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)],
            INF
        ); 

        if (using_k_path < cell_i_j) {
            cell_i_j = using_k_path;
        }

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = cell_i_j;
}




In [84]:
%%file device_floyd_warshall_v_3_0.cu

#include "include/include_needed_libraries.cuh"
#include "include/lcm.hpp"

#define ARR_MATRIX_INDEX(i,j,n) (i*n+j)
#define ARR_MATRIX_INDEX_TRASP(i,j,n) (i+n*j)

#define SHARED_BANK_N_INT 32
#define ARR_MATRIX_INDEX_BANK_CONFLICT(i, j, n, handle_bank_conflict) (i*n + j + (handle_bank_conflict ? i : 0))
#define ARR_MATRIX_SIZE_BANK_CONFICT(B,handle_bank_conflict) (B*B + (handle_bank_conflict ? (B-1) : 0))

//main device code
void floyd_warshall_blocked_device_v_3_0(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_3_0_phase_1(int *matrix, int n, int t, bool handle_bank_conflict);

__global__ void execute_round_device_v_3_0_phase_2_row_portion(int *matrix, int n, int t, int start_col);
__global__ void execute_round_device_v_3_0_phase_2_col_portion(int *matrix, int n, int t, int start_row);
__global__ void execute_round_device_v_3_0_phase_3_portion(int *matrix, int n, int t, int start_row, int start_col);


int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_3_0);
}

void floyd_warshall_blocked_device_v_3_0(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed max block size

    cudaStream_t streams[4];
    for (int i=0; i<4; i++) {
        cudaStreamCreate(&streams[i]);
    }

    int *dev_rand_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_rand_matrix, n*n*sizeof(int)));
    HANDLE_ERROR(cudaMemcpy(dev_rand_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    int num_rounds = n/B; 

    bool bank_conflict_phase_1 = lcm(SHARED_BANK_N_INT, B) <= (B-1)*B;
     
    for(int t = 0; t < num_rounds; t++) { 

        //arr_execute_round(int *matrix, int n, int t, int row, int col, int B)

        //phase 1: self-dependent block
        dim3 num_blocks_phase_1(1, 1);
        dim3 threads_per_block_phase_1(B, B);

        execute_round_device_v_3_0_phase_1<<<
            num_blocks_phase_1, 
            threads_per_block_phase_1, 
            ARR_MATRIX_SIZE_BANK_CONFICT(B, bank_conflict_phase_1)*sizeof(int), 
            streams[0]>>>(dev_rand_matrix, n, t, bank_conflict_phase_1);

        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 2: all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        // dim3 num_blocks_phase_2(1, num_rounds-1);  

        // execute_round_device_v_3_0_phase_2_row<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);
        // execute_round_device_v_3_0_phase_2_col<<<num_rounds-1, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);

        // up 
        execute_round_device_v_3_0_phase_2_col_portion<<<
            t, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[0]>>>(dev_rand_matrix, n, t, 0);

        // left
        execute_round_device_v_3_0_phase_2_row_portion<<<
            t, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[1]>>>(dev_rand_matrix, n, t, 0);

        // down
        execute_round_device_v_3_0_phase_2_col_portion<<<
            num_rounds-1-t, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[2]>>>(dev_rand_matrix, n, t, t+1);

        // right
        execute_round_device_v_3_0_phase_2_row_portion<<<
            num_rounds-1-t, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[3]>>>(dev_rand_matrix, n, t, t+1);


        HANDLE_ERROR(cudaDeviceSynchronize());

        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t

        // dim3 num_blocks_phase_3(num_rounds-1, num_rounds-1); 
        // execute_round_device_v_3_0_phase_3<<<num_blocks_phase_3, threads_per_block_phase_1, 2*B*B*sizeof(int)>>>(dev_rand_matrix, n, t);

        dim3 num_blocks_phase_3_ul(t, t);
        execute_round_device_v_3_0_phase_3_portion<<<
            num_blocks_phase_3_ul, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[0]>>>(dev_rand_matrix, n, t, 0, 0);

        dim3 num_blocks_phase_3_dr(num_rounds-t-1, num_rounds-t-1); 
        execute_round_device_v_3_0_phase_3_portion<<<
            num_blocks_phase_3_dr, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[1]>>>(dev_rand_matrix, n, t, t+1, t+1);

        dim3 num_blocks_phase_3_ur(t, num_rounds-t-1); 
        execute_round_device_v_3_0_phase_3_portion<<<
            num_blocks_phase_3_ur, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[2]>>>(dev_rand_matrix, n, t, 0, t+1);

        dim3 num_blocks_phase_3_dl(num_rounds-t-1, t); 
        execute_round_device_v_3_0_phase_3_portion<<<
            num_blocks_phase_3_dl, threads_per_block_phase_1, 
            2*B*B*sizeof(int), 
            streams[3]>>>(dev_rand_matrix, n, t, t+1, 0);

        HANDLE_ERROR(cudaDeviceSynchronize()); 
    }

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaMemcpy(matrix, dev_rand_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));
    HANDLE_ERROR(cudaFree(dev_rand_matrix));

    for (int i=0; i<4; i++) {
        HANDLE_ERROR(cudaStreamDestroy(streams[i]));
    }
}


__global__ void execute_round_device_v_3_0_phase_1(int *matrix, int n, int t, bool handle_bank_conflict) {

    // Launched block and correspondent position in the matrix

    //  t

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    extern __shared__ int block_t_t_shared[];

    int i = threadIdx.x + t * blockDim.x;  // row abs index
    int j = threadIdx.y + t * blockDim.x;  // col abs index

    block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, k, blockDim.x, handle_bank_conflict)], 
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(k, threadIdx.y, blockDim.x, handle_bank_conflict)], 
            INF
        ); 

        if (using_k_path < block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)]) {
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = using_k_path;
        }
        
        __syncthreads();
    }

    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)];
}


__global__ void execute_round_device_v_3_0_phase_2_row_portion(int *matrix, int n, int t, int start_col) {
extern __shared__ int shared_mem[];
    
    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[(blockDim.x * blockDim.x)];

    // it's a row ...

    // abs row index 
    int i = BLOCK_START(t, blockDim.x) + threadIdx.x;    
    // abs col index   
    int j = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.y + start_col * blockDim.x; 

    // the block where I am working
    block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round (transposed to avoid bank conflict)
    block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];


    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        // Because we are doing rows:
        // -    matrix[i,abs_k] is in block_t_t_shared[threadIdx.x,k]
        // -    matrix[abs_k,j] is in block_i_j_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_i_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)];
}


__global__ void execute_round_device_v_3_0_phase_2_col_portion(int *matrix, int n, int t, int start_row) {
extern __shared__ int shared_mem[];

    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[blockDim.x*blockDim.x];

    // it's a column ...

    // abs row index 
    int i = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.x + start_row * blockDim.x;
    // abs col index 
    int j = BLOCK_START(t, blockDim.x) + threadIdx.y;

    // the block where I am working (transposed to avoid bank conflict)
    block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round 
    block_t_t_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {
        
        // Because we are doing columns:
        // -    matrix[i,k] is in block_i_j_shared[threadIdx.x,k]
        // -    matrix[k,j] is in block_t_t_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_t_t_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)];
}



__global__ void execute_round_device_v_3_0_phase_3_portion(int *matrix, int n, int t, int start_row, int start_col) {

    extern __shared__ int shared_mem[];

    int* block_i_t_shared = &shared_mem[0];
    int* block_t_j_shared = &shared_mem[blockDim.x*blockDim.x];

    // abs row index
    int i = threadIdx.x + blockIdx.x * blockDim.x + start_row * blockDim.x;
    // abs col index
    int j = threadIdx.y + blockIdx.y * blockDim.y + start_col * blockDim.y;

    // printf("%d,%d\n",i,j);
    
    // since the cell i,j is read and written only by this thread
    // there is no need to copy its value to shared memory we can just us a local variable
    int cell_i_j = matrix[ARR_MATRIX_INDEX(i, j, n)];
        
    // In phase 3 I copy in two portions of my shared memory
    // the block corresponding to (t, this column) and (this row, t). 

    // (this row, t) is transposed to prevent bank conflict

    block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(i, (BLOCK_START(t, blockDim.x) + threadIdx.y), n)
    ];
    block_t_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX((BLOCK_START(t, blockDim.x) + threadIdx.x), j, n)
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)],
            block_t_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)],
            INF
        ); 

        if (using_k_path < cell_i_j) {
            cell_i_j = using_k_path;
        }

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = cell_i_j;
}



In [85]:
%%file device_floyd_warshall_v_3_1.cu

#include "include/include_needed_libraries.cuh"
#include "include/lcm.hpp"
#include <vector>

#define ARR_MATRIX_INDEX(i,j,n) (i*n+j)
#define ARR_MATRIX_INDEX_TRASP(i,j,n) (i+n*j)

#define SHARED_BANK_N_INT 32
#define ARR_MATRIX_INDEX_BANK_CONFLICT(i, j, n, handle_bank_conflict) (i*n + j + (handle_bank_conflict ? i : 0))
#define ARR_MATRIX_SIZE_BANK_CONFICT(B,handle_bank_conflict) (B*B + (handle_bank_conflict ? (B-1) : 0))

//main device code
void floyd_warshall_blocked_device_v_3_1(int *matrix, int n, int B);

//rounds code
__global__ void execute_round_device_v_3_1_phase_1(int *matrix, int n, int t, bool handle_bank_conflict);

__global__ void execute_round_device_v_3_1_phase_2_row_portion(int *matrix, int n, int t, int start_col, int end_col);
__global__ void execute_round_device_v_3_1_phase_2_col_portion(int *matrix, int n, int t, int start_row, int end_row);
__global__ void execute_round_device_v_3_1_phase_3_portion(int *matrix, int n, int t, int start_row, int start_col, int end_row, int end_col);


int main(int argc, char *argv[]) {

    return handle_arguments_and_execute(argc, argv, (void(*) (int*, int, int)) &floyd_warshall_blocked_device_v_3_1);
}

cudaKernelNodeParams cuda_graph_node_params_copy(cudaKernelNodeParams params) {
    
    cudaKernelNodeParams newParams = { 0 };

    newParams.func = params.func;
    newParams.blockDim = params.blockDim;
    newParams.gridDim = params.gridDim;
    newParams.kernelParams = params.kernelParams;
    newParams.sharedMemBytes = params.sharedMemBytes;
    newParams.extra = params.extra;

    return newParams;
}


void floyd_warshall_blocked_device_v_3_1(int *matrix, int n, int B) {

    assert(n%B == 0);                       // B must divide n
    assert(B*B<=MAX_BLOCK_SIZE);            // B*B cannot exceed max block size

    // init the graph i will use to do all rounds
    cudaGraph_t graph;
    cudaGraphCreate(&graph, 0);
    std::vector<cudaGraphNode_t> nodeDependencies = {}; // Dependency vector 

    int *dev_matrix;
    HANDLE_ERROR(cudaMalloc( (void**) &dev_matrix, n*n*sizeof(int)));

    // HANDLE_ERROR(cudaMemcpy(dev_matrix, matrix, n*n*sizeof(int), cudaMemcpyHostToDevice));

    cudaMemcpy3DParms copy_host_to_dev_params = {0};

    copy_host_to_dev_params.srcArray = NULL;
    copy_host_to_dev_params.srcPos = make_cudaPos(0, 0, 0);
    copy_host_to_dev_params.srcPtr = make_cudaPitchedPtr((void*) matrix, n*n*sizeof(int), n*n, 1);
    copy_host_to_dev_params.dstArray = NULL;
    copy_host_to_dev_params.dstPos = make_cudaPos(0, 0, 0);
    copy_host_to_dev_params.dstPtr = make_cudaPitchedPtr((void*) dev_matrix, n*n*sizeof(int), n*n, 1);
    copy_host_to_dev_params.extent = make_cudaExtent(n*n*sizeof(float), 1, 1);
    copy_host_to_dev_params.kind = cudaMemcpyHostToDevice;

    cudaGraphNode_t copy_host_to_dev_node;

    HANDLE_ERROR(cudaGraphAddMemcpyNode(
        &copy_host_to_dev_node, graph, 
        nodeDependencies.data(), nodeDependencies.size(), 
        &copy_host_to_dev_params
        ));


    // number of rounds that will be executed
    int num_rounds = n/B;

    // check if there will be bank conflict in phase 1
    bool bank_conflict_phase_1 = lcm(SHARED_BANK_N_INT, B) <= (B-1)*B;

    // number of threads launched at each kernel 
    // (it has to be the same number because of the cuda graphs, 
    // the exceeding threads will end as firtst instruction)
    dim3 num_blocks(max(num_rounds-1, 1), max(num_rounds-1, 1));
    dim3 threads_per_block(B, B);

    // a variable needed for calls which requires pointer args
    int zero = 0;

    // previous round nodes (so i can add them as dependency for the next)
    cudaGraphNode_t prev_phase3_up_left_node,   prev_phase3_up_right_node, 
                    prev_phase3_down_left_node, prev_phase3_down_right_node;
         
    for(int t = 0; t < num_rounds; t++) { 

        // a variable needed for calls which requires pointer args
        int t_plus_1 = t+1;

        // ----------------------------------------------------------------------
        // phase 1: self-dependent block

        // execute_round_device_v_3_1_phase_1<<<
        //     num_blocks, 
        //     threads_per_block, 
        //     ARR_MATRIX_SIZE_BANK_CONFICT(B, bank_conflict_phase_1)*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, bank_conflict_phase_1);

        // HANDLE_ERROR(cudaDeviceSynchronize());

        void* phase1_args[4] = { (void*) &dev_matrix, (void*) &n, (void*) &t, (void*) &bank_conflict_phase_1 };

        cudaKernelNodeParams phase1_params;

        phase1_params.func = (void*) execute_round_device_v_3_1_phase_1;
        phase1_params.gridDim = num_blocks;
        phase1_params.blockDim = threads_per_block;
        phase1_params.sharedMemBytes = max(
            ARR_MATRIX_SIZE_BANK_CONFICT(B, bank_conflict_phase_1)*sizeof(int), 
            2*B*B*sizeof(int)
        );
        phase1_params.sharedMemBytes = ARR_MATRIX_SIZE_BANK_CONFICT(B, bank_conflict_phase_1)*sizeof(int);
        phase1_params.kernelParams = (void**) phase1_args;
        phase1_params.extra = NULL;

        nodeDependencies.clear(); 
        if (t > 0) {
            // round after first should depend from previous
            nodeDependencies.push_back(prev_phase3_up_left_node);
            nodeDependencies.push_back(prev_phase3_up_right_node);
            nodeDependencies.push_back(prev_phase3_down_right_node);
            nodeDependencies.push_back(prev_phase3_down_left_node);
        } else {
            // first round depends 
            nodeDependencies.push_back(copy_host_to_dev_node);
        }

        cudaGraphNode_t phase1_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase1_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase1_params));

        // HANDLE_ERROR(cudaDeviceSynchronize());

        // ----------------------------------------------------------------------
        // phase 2: row and cols
        // all blocks that share a row or a column with the self dependent, so
        //  -   all blocks just above or under t
        //  -   all block at left and at right of t

        nodeDependencies.clear();
        nodeDependencies.push_back(phase1_node);

        // up 
        // execute_round_device_v_3_1_phase_2_col_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, 0, t);

        void* phase2_up_left_args[5] = { (void*) &dev_matrix, 
            &n, &t, &zero, &t };

        cudaKernelNodeParams phase2_up_params = cuda_graph_node_params_copy(phase1_params);

        phase2_up_params.func = (void*) execute_round_device_v_3_1_phase_2_col_portion;
        phase2_up_params.sharedMemBytes = 2*B*B*sizeof(int);
        phase2_up_params.kernelParams = (void**) phase2_up_left_args;

        cudaGraphNode_t phase2_up_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase2_up_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase2_up_params
        ));

        // left
        // execute_round_device_v_3_1_phase_2_row_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, 0, t);

        cudaKernelNodeParams phase2_left_params = cuda_graph_node_params_copy(phase2_up_params);
        phase2_left_params.func = (void*) execute_round_device_v_3_1_phase_2_row_portion;

        cudaGraphNode_t phase2_left_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase2_left_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase2_left_params));

        // down
        // execute_round_device_v_3_1_phase_2_col_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, t+1, num_blocks);

        cudaKernelNodeParams phase2_down_params = cuda_graph_node_params_copy(phase2_up_params);
        void* phase2_down_right_args[5] = { (void*) &dev_matrix, 
            &n, &t, &t_plus_1, &num_rounds};
        phase2_down_params.kernelParams = (void**) phase2_down_right_args;

        cudaGraphNode_t phase2_down_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase2_down_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase2_down_params));

        // right
        // execute_round_device_v_3_1_phase_2_row_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, t+1, num_blocks);

        cudaKernelNodeParams phase2_right_params = cuda_graph_node_params_copy(phase2_down_params);
        phase2_right_params.func = (void*) execute_round_device_v_3_1_phase_2_row_portion;

        cudaGraphNode_t phase2_right_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase2_right_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase2_right_params));

        // HANDLE_ERROR(cudaDeviceSynchronize());
        
        // phase 3: all the remaining blocks, so all the blocks that don't share a row or a col with t
        
        // up-left
        // execute_round_device_v_3_1_phase_3_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, 0, 0, t, t);

        void* phase3_up_left_args[7] = {(void*) &dev_matrix, 
            &n, &t, &zero, &zero, &t, &t};

        cudaKernelNodeParams phase3_up_left_params = cuda_graph_node_params_copy(phase2_up_params);

        phase3_up_left_params.func = (void*) execute_round_device_v_3_1_phase_3_portion;
        phase3_up_left_params.kernelParams = (void**) phase3_up_left_args;

        nodeDependencies.clear();
        nodeDependencies.push_back(phase2_up_node);
        nodeDependencies.push_back(phase2_left_node);

        cudaGraphNode_t phase3_up_left_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase3_up_left_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase3_up_left_params));

        // up-right
        // execute_round_device_v_3_1_phase_3_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, 0, t+1, t, num_rounds);

        void* phase3_up_right_args[7] = {(void*) &dev_matrix, 
            &n, &t, &zero, &t_plus_1, &t, &num_rounds};

        cudaKernelNodeParams phase3_up_right_params = cuda_graph_node_params_copy(phase3_up_left_params);
        phase3_up_right_params.kernelParams = (void**) phase3_up_right_args;

        nodeDependencies.clear();
        nodeDependencies.push_back(phase2_up_node);
        nodeDependencies.push_back(phase2_right_node);

        cudaGraphNode_t phase3_up_right_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase3_up_right_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase3_up_right_params));

        // down-right
        // execute_round_device_v_3_1_phase_3_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, t+1, t+1, num_rounds, num_rounds);

        void* phase3_down_right_args[7] = {(void*) &dev_matrix, 
            &n, &t, &t_plus_1, &t_plus_1, &num_rounds, &num_rounds};

        cudaKernelNodeParams phase3_down_right_params = cuda_graph_node_params_copy(phase3_up_left_params);
        phase3_down_right_params.kernelParams = (void**) phase3_down_right_args;

        nodeDependencies.clear();
        nodeDependencies.push_back(phase2_down_node);
        nodeDependencies.push_back(phase2_right_node);

        cudaGraphNode_t phase3_down_right_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase3_down_right_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase3_down_right_params));

        // down-left
        // execute_round_device_v_3_1_phase_3_portion<<<
        //     num_blocks, threads_per_block, 
        //     2*B*B*sizeof(int), 
        //     graph_stream>>>(dev_matrix, n, t, t+1, 0, num_rounds, t);
        
        void* phase3_down_left_args[7] = {(void*) &dev_matrix, 
            &n, &t, &t_plus_1, &zero, &num_rounds, &t};

        cudaKernelNodeParams phase3_down_left_params = cuda_graph_node_params_copy(phase3_up_left_params);
        phase3_down_left_params.kernelParams = (void**) phase3_down_left_args;

        nodeDependencies.clear();
        nodeDependencies.push_back(phase2_down_node);
        nodeDependencies.push_back(phase2_left_node);

        cudaGraphNode_t phase3_down_left_node;

        HANDLE_ERROR(cudaGraphAddKernelNode(
            &phase3_down_left_node, graph, 
            nodeDependencies.data(), nodeDependencies.size(), 
            &phase3_down_left_params));

        // HANDLE_ERROR(cudaDeviceSynchronize());   

        // save phase 3 nodes
        prev_phase3_up_left_node    = phase3_up_left_node;
        prev_phase3_up_right_node   = phase3_up_right_node;
        prev_phase3_down_right_node = phase3_down_right_node;
        prev_phase3_down_left_node  = phase3_down_left_node;
    }

    // Add copy of final result from device to host (as graph)
    // HANDLE_ERROR(cudaMemcpy(matrix, dev_matrix, n*n*sizeof(int), cudaMemcpyDeviceToHost));

    cudaMemcpy3DParms copy_dev_to_host_params = {0};

    copy_dev_to_host_params.srcArray = NULL;
    copy_dev_to_host_params.srcPos = make_cudaPos(0, 0, 0);
    copy_dev_to_host_params.srcPtr = make_cudaPitchedPtr((void*) dev_matrix, n*n*sizeof(int), n*n, 1);
    copy_dev_to_host_params.dstArray = NULL;
    copy_dev_to_host_params.dstPos = make_cudaPos(0, 0, 0);
    copy_dev_to_host_params.dstPtr = make_cudaPitchedPtr((void*) matrix, n*n*sizeof(int), n*n, 1);
    copy_dev_to_host_params.extent = make_cudaExtent(n*n*sizeof(float), 1, 1);
    copy_dev_to_host_params.kind = cudaMemcpyDeviceToHost;

    nodeDependencies.clear();
    nodeDependencies.push_back(prev_phase3_up_left_node);
    nodeDependencies.push_back(prev_phase3_up_right_node);
    nodeDependencies.push_back(prev_phase3_down_right_node);
    nodeDependencies.push_back(prev_phase3_down_left_node);

    cudaGraphNode_t copy_dev_to_host_node;

    HANDLE_ERROR(cudaGraphAddMemcpyNode(
        &copy_dev_to_host_node, graph, 
        nodeDependencies.data(), nodeDependencies.size(), 
        &copy_dev_to_host_params
        ));

    // stream used for executing graph
    cudaStream_t graph_stream;
    cudaStreamCreate(&graph_stream);

    // Instanciate and run graph
    cudaGraphExec_t instance;
    HANDLE_ERROR(cudaGraphInstantiate(&instance, graph, NULL, NULL, 0));
    HANDLE_ERROR(cudaGraphLaunch(instance, graph_stream));
    HANDLE_ERROR(cudaStreamSynchronize(graph_stream));

    // Clean up
    HANDLE_ERROR(cudaGraphExecDestroy(instance));
    HANDLE_ERROR(cudaGraphDestroy(graph));

    // HANDLE_ERROR(cudaDeviceSynchronize());  

    HANDLE_ERROR(cudaFree(dev_matrix));

    HANDLE_ERROR(cudaStreamDestroy(graph_stream));

}


__global__ void execute_round_device_v_3_1_phase_1(int *matrix, int n, int t, bool handle_bank_conflict) {

    // Launched block and correspondent position in the matrix

    //  t   -   -   -   -
    //  -   -   -   -   -
    //  -   -   -   -   -
    //  -   -   -   -   -
    //  -   -   -   -   -
    

    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 
    //  .   .   .   t   .   .
    //  .   .   .   .   .   . 
    //  .   .   .   .   .   . 

    if (blockIdx.x > 0 || blockIdx.y > 0)   return;

    // if (threadIdx.x == 0 && threadIdx.y == 0) printf("(%d,%d) ", blockIdx.x, blockIdx.y);

    extern __shared__ int block_t_t_shared[];

    int i = threadIdx.x + t * blockDim.x;  // row abs index
    int j = threadIdx.y + t * blockDim.x;  // col abs index

    block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, k, blockDim.x, handle_bank_conflict)], 
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(k, threadIdx.y, blockDim.x, handle_bank_conflict)], 
            INF
        ); 

        if (using_k_path < block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)]) {
            block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)] = using_k_path;
        }
        
        __syncthreads();
    }

    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_t_t_shared[ARR_MATRIX_INDEX_BANK_CONFLICT(threadIdx.x, threadIdx.y, blockDim.x, handle_bank_conflict)];
}


__global__ void execute_round_device_v_3_1_phase_2_row_portion(int *matrix, int n, int t, int start_col, int end_col) {
    
    if (blockIdx.x >= end_col-start_col)    return;
    
    extern __shared__ int shared_mem[];
    
    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[(blockDim.x * blockDim.x)];

    // it's a row ...

    // abs row index 
    int i = BLOCK_START(t, blockDim.x) + threadIdx.x;    
    // abs col index   
    int j = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.y + start_col * blockDim.x; 

    // the block where I am working
    block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round (transposed to avoid bank conflict)
    block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];


    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        // Because we are doing rows:
        // -    matrix[i,abs_k] is in block_t_t_shared[threadIdx.x,k]
        // -    matrix[abs_k,j] is in block_i_j_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_t_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_i_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)];
}


__global__ void execute_round_device_v_3_1_phase_2_col_portion(int *matrix, int n, int t, int start_row, int end_row) {
    
    if (blockIdx.x >= end_row-start_row)    return;
    
    extern __shared__ int shared_mem[];

    int* block_i_j_shared = &shared_mem[0];
    int* block_t_t_shared = &shared_mem[blockDim.x*blockDim.x];

    // it's a column ...

    // abs row index 
    int i = BLOCK_START(blockIdx.x, blockDim.x) + threadIdx.x + start_row * blockDim.x;
    // abs col index 
    int j = BLOCK_START(t, blockDim.x) + threadIdx.y;

    // the block where I am working (transposed to avoid bank conflict)
    block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[ARR_MATRIX_INDEX(i, j, n)];

    // the self-dependent block already calculated in this round 
    block_t_t_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(
            (BLOCK_START(t, blockDim.x) + threadIdx.x), 
            (BLOCK_START(t, blockDim.x) + threadIdx.y), 
            n
        )
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {
        
        // Because we are doing columns:
        // -    matrix[i,k] is in block_i_j_shared[threadIdx.x,k]
        // -    matrix[k,j] is in block_t_t_shared[k,threadIdx.y]
        int using_k_path = sum_if_not_infinite(
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)], 
            block_t_t_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)], 
            INF
        ); 

        if (using_k_path < block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)]) {
            block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = using_k_path;
        }

        //printf("i:%d, j:%d, k:%d\n", i, j, k);

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = block_i_j_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)];
}



__global__ void execute_round_device_v_3_1_phase_3_portion(int *matrix, int n, int t, int start_row, int start_col, int end_row, int end_col) {

    if (blockIdx.x >= end_row-start_row || blockIdx.y >= end_col-start_col)    return;
    
    extern __shared__ int shared_mem[];

    int* block_i_t_shared = &shared_mem[0];
    int* block_t_j_shared = &shared_mem[blockDim.x*blockDim.x];

    // abs row index
    int i = threadIdx.x + blockIdx.x * blockDim.x + start_row * blockDim.x;
    // abs col index
    int j = threadIdx.y + blockIdx.y * blockDim.y + start_col * blockDim.y;

    // printf("%d,%d\n",i,j);
    
    // since the cell i,j is read and written only by this thread
    // there is no need to copy its value to shared memory we can just us a local variable
    int cell_i_j = matrix[ARR_MATRIX_INDEX(i, j, n)];
        
    // In phase 3 I copy in two portions of my shared memory
    // the block corresponding to (t, this column) and (this row, t). 

    // (this row, t) is transposed to prevent bank conflict

    block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX(i, (BLOCK_START(t, blockDim.x) + threadIdx.y), n)
    ];
    block_t_j_shared[ARR_MATRIX_INDEX(threadIdx.x, threadIdx.y, blockDim.x)] = matrix[
        ARR_MATRIX_INDEX((BLOCK_START(t, blockDim.x) + threadIdx.x), j, n)
    ];
    
    __syncthreads();

    // now k is iterating the relative indexind of (t,t) block 
    // in shared memory (instead of the abs position in matrix)
    for (int k = 0; k < blockDim.x; k++) {

        int using_k_path = sum_if_not_infinite(
            block_i_t_shared[ARR_MATRIX_INDEX_TRASP(threadIdx.x, k, blockDim.x)],
            block_t_j_shared[ARR_MATRIX_INDEX(k, threadIdx.y, blockDim.x)],
            INF
        ); 

        if (using_k_path < cell_i_j) {
            cell_i_j = using_k_path;
        }

        __syncthreads();
    }

    // copy result in global memory
    matrix[ARR_MATRIX_INDEX(i, j, n)] = cell_i_j;
}



In [121]:
! make dev VERSION=1_3_pitch

In [122]:
 ! ./bin/fwa_dev_v_1_3_pitch.out test

In [123]:
! nvprof --profile-from-start on ./bin/fwa_dev_v_1_3_pitch.out perf -n=50 -b=10 -t=1

In [89]:
! nvprof --csv --log-file csv/out.csv --normalized-time-unit us --profile-from-start off ./bin/fwa_dev_v_3_0.out perf -n=50 -b=10 -t=1

In [90]:
! cat csv/out.csv

In [91]:
%%file generate_and_print_n_b.cpp

#include "include/generate_n_b_couples.hpp"
#include "include/macros.hpp"

#include "time.h"

int main(int argc, char** argv) {

    int seed = time(NULL);
    srand(seed);

    // define parameters
    double to_multiply = 1;
    int to_sum = 80;
    int min_input_size = 80;
    int max_input_size = 480;

    //if blocking factors are calculated randomly
    int min_blocking_factor = 2;
    int max_num_of_b_per_n = 5;
    int max_num_tests = 50;
    //if blocking factors are given from argv
    int* B;
    if (argc > 1) {
        B = (int *) malloc(sizeof(int) * (argc-1));
    }

    std::vector<std::pair<int, int>> list_of_all_n_b;

    // print parameters
    printf("to_multiply:\t\t%f\n", to_multiply);
    printf("to_sum:\t\t\t%d\n", to_sum);
    printf("seed:\t\t\t%d\n", seed);
    printf("min_input_size:\t\t%d\n", min_input_size);
    printf("max_input_size:\t\t%d\n", max_input_size);

    if (argc == 1) {
        // generate blocking factors randomly
        printf("min_blocking_factor:\t%d\n", min_blocking_factor);
        printf("max_num_of_b_per_n:\t%d\n", max_num_of_b_per_n);
        printf("max_num_tests:\t\t%d\n", max_num_tests);
        
        // generate list randomly
        list_of_all_n_b = generate_list_of_all_n_b(min_input_size, max_input_size, max_num_of_b_per_n, to_multiply, to_sum, min_blocking_factor, max_num_tests, seed);

    } else {
        // read blocking factors from argv
        for (int i = 1; i < argc; i++) {
            B[i-1] = atoi(argv[i]);
        }
        for (int n = min_input_size; n <= max_input_size; n = mmax(((int) (to_multiply * (double) n)) + to_sum, (n+1)) ) {
            for (int i = 0; i < argc-1; i++) {
                if (n % B[i] == 0) {
                    list_of_all_n_b.push_back(std::make_pair(n, B[i]));
                }   
            }
        }
    }


    // define output file name
    std::string out_filename = "csv/list_of_n_b.csv";

    // print list to file
    print_list_to_file(list_of_all_n_b, out_filename);

    // print file name
    printf("\noutput file:\t\t%s\n", out_filename.c_str());

    return 0;
    
}

In [92]:
# ! make generate_n_b
# ! echo
# ! bin/generate_and_print_n_b.out 8 16 24 32
# ! echo
# ! cat csv/list_of_n_b.csv

In [93]:
! rm -r csv
! mkdir csv

In [94]:
import re #regex
import os #operating system
import pandas as pd
import random

# INPUT TEST DIMENSIONS

# define and print command for compiling test dimension cpp file
make_test_dim_cmd = "make generate_n_b"
print(make_test_dim_cmd)
# compile test dimension cpp file to bin file
os.system(make_test_dim_cmd)
print()

# generate all test dimensions
os.system("bin/generate_and_print_n_b.out 8 16 24 32")
print()

# store test dimensions to csv and print them
test_dimensions = pd.read_csv('csv/list_of_n_b.csv')
print("test_dimensions:")
print(test_dimensions)
print()

# -----------------------------------------------------------------

# ALGORITHM EXECUTION

cuda_files = os.listdir()
#print(files)

# calculate and print the list of all random seeds
#all_seeds = []
#print("seeds:")
#for i in range(len(test_dimensions)) :
#    rand = random.randint(0,9999999)
#    all_seeds.append(rand)
#    print(i, rand)
#print()

# use just one seed
rand_seed = random.randint(0,9999999)


# test each version
for file in cuda_files :

    if re.match("device_floyd_warshall_v_.*\.cu", file) and (not re.match("device_floyd_warshall_v_.*_ERROR\.cu", file)):

        # is a floyd_warshall cuda file

        # obtain cuda file version
        version = re.sub("^device_floyd_warshall_v_", "", file)
        version = re.sub("\.cu$", "", version)
        
        # define floyd warshall bin file
        fw_bin = 'bin/fwa_dev_v_' + version + '.out'
        
        # print version, cuda file name, bin file name
        print(f"version:   {version}")
        print(f"cuda file: {file}")
        print(f"bin file:  {fw_bin}\n")

        # define and print command for compiling cuda file to bin file
        make_algorithm_cmd = "make dev VERSION=" + version
        print(make_algorithm_cmd)
        # compile test dimension cpp file to bin file
        os.system(make_algorithm_cmd)
        print()

        # define parameters
        #  - exec option: {'perf', 'test'}
        exec_option='perf'
        #  - number of tests for each couple (n,b)
        t=1

        for row in test_dimensions.iterrows() :

            # foreach test dimension couple (n,b) :

            i = row[0]
            n = row[1][0]
            b = row[1][1]
            #print(n, b)
            
            csv_output = 'csv/fwa_dev_v_' + str(version) + '__n_' + str(n).zfill(3) + '__b_' + str(b).zfill(2) + "__t_" + str(t).zfill(2) + ".csv"
            print(f"out file {i:02}: {csv_output}")

            launch_cmd = "nvprof --csv --log-file " + csv_output + " --normalized-time-unit us --profile-from-start off ./" + fw_bin + " " + exec_option + " -t=" + str(t) + " -n=" + str(n) + " -b=" + str(b) + " -s=" + str(rand_seed) #str(all_seeds[i])
            print(launch_cmd)
            os.system(launch_cmd)
            print()

        print("-----------------------------------------------------------------")
        print()


In [95]:
! ls -lh csv

In [96]:
import shutil
shutil.make_archive('all_csv_output', 'zip', 'csv')